In [ ]:
import itertools
import json
import pickle
import re
from itertools import cycle
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scri
from matplotlib import cycler
from scipy.interpolate import InterpolatedUnivariateSpline as Spline
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from spherical_functions import LM_index as lm

plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (12, 10)
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
line_styles = ["-", "--", "-.", ":"]
combined_cycler = cycler(linestyle=line_styles) * cycler(color=colors)
plt.rcParams["axes.prop_cycle"] = combined_cycler


# Functions

In [ ]:
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len

def load_and_pickle(data_path:Path, reload_data:bool=False, data_type:str='abd', options:dict={}):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")

  saved_data_path = data_path.parent/"saved.pkl"
  
  if saved_data_path.exists() and reload_data == False:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      print(f"Saved data loaded: {saved_data_path}")
  else:
    saved_data = {}
    if data_type == 'abd':
      saved_data['abd']= scri.create_abd_from_h5(
          file_name=str(data_path),
          file_format="spectrecce_v1",
          **options
        )
      with open(saved_data_path, 'wb') as f:
        pickle.dump(saved_data,f)
      print(f"Data loaded and saved at : {saved_data_path}")

  return saved_data

def load_bondi_constraints(data_path:Path):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")
  saved_data_path = data_path.parent/"saved.pkl"
  if not saved_data_path.exists():
    raise Exception(f"{saved_data_path} does not exist")
  else:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      if 'bondi_violation_norms' in saved_data:
        print(f"bondi_violation_norms loaded for {data_path}")
      else:
        print(f"Computing bondi_violation_norms for: {data_path}")
        saved_data['bondi_violation_norms'] = saved_data['abd'].bondi_violation_norms
        with open(saved_data_path, 'wb') as f:
          pickle.dump(saved_data,f)

        print(f"Saved bondi_violation_norms for: {data_path}")
    return saved_data

def add_bondi_constraints(abd_data:dict):
  for key in abd_data:
    abd_data[key]['bondi_violation_norms'] = abd_data[key]["abd"].bondi_violation_norms
    print(f"bondi_violation_norms computed for {key}")

def create_diff_dict_cce(WT_data_dict:dict, l:int, m:int, base_key:str, t_interpolate:np.ndarray):
  h = WT_data_dict[base_key]['abd'].h.interpolate(t_interpolate)
  diff_dict = {"t": h.t}
  y_base = h.data[:,  lm(l,m,h.ell_min)]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    h = WT_data_dict[key]['abd'].h.interpolate(t_interpolate)
    y_inter = h.data[:,  lm(l,m,h.ell_min)]
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

def extract_radii(h5_file_path:Path):
  radii = set()
  with h5py.File(h5_file_path,'r') as f:
    names = []
    f.visit(names.append)
  for name in names:
    if "Version" in name:
      continue
    radii.add(name[1:5])
  radii = list(radii)
  radii.sort()
  return radii


def generate_columns(num_cols:int,beta_type=False):
  if beta_type:
    num_cols = num_cols*2
  L_max = int(np.sqrt((num_cols-1)/2))-1
  # print(L_max,np.sqrt((num_cols-1)/2)-1)
  col_names = ['t(M)']
  for l in range(0,L_max+1):
    for m in range(-l,l+1):
      if beta_type:
        if m==0:
          col_names.append(f"Re({l},{m})")
        elif m < 0:
          continue
        else:
          col_names.append(f"Re({l},{m})")
          col_names.append(f"Im({l},{m})")
      else:
        col_names.append(f"Re({l},{m})")
        col_names.append(f"Im({l},{m})")
  return col_names


def WT_to_pandas(horizon_path:Path, keys_to_load:list=None):
    assert(horizon_path.exists())
    df_dict = {}
    beta_type_list = ['Beta.dat', 'DuR.dat', 'R.dat', 'W.dat']
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if keys_to_load is not None:
                if key not in keys_to_load:
                    continue
            if key == "VersionHist.ver":
              continue 
            if key in beta_type_list:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1],beta_type=True))
            else:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1]))


    return df_dict


def create_diff_dict(WT_data_dict:dict, mode:str, variable:str, base_key:str):
  diff_dict = {"t(M)":WT_data_dict[base_key][variable]['t(M)']}
  y_base = WT_data_dict[base_key][variable][mode]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    y = WT_data_dict[key][variable][mode]
    t = WT_data_dict[key][variable]['t(M)']
    y_interpolator = interp1d(t, y, kind='cubic',fill_value='extrapolate')
    y_inter = y_interpolator(diff_dict['t(M)'])
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

def filter_by_regex(regex,col_list,exclude=False):
  filtered_set = set()
  if type(regex) is list:
    for reg in regex:
      for i in col_list:
        if re.search(reg,i):
          filtered_set.add(i)
  else:
    for i in col_list:
      if re.search(regex,i):
        filtered_set.add(i)

  filtered_list = list(filtered_set)
  if exclude:
    col_list_copy = list(col_list.copy())
    for i in filtered_list:
      if i in col_list_copy:
        col_list_copy.remove(i)
    filtered_list = col_list_copy

  # Restore the original order
  filtered_original_ordered_list = []
  for i in list(col_list):
    if i in filtered_list:
      filtered_original_ordered_list.append(i)
  return filtered_original_ordered_list

def limit_by_col_val(min_val,max_val,col_name,df):
  filter = (df[col_name]>=min_val) &(df[col_name] <=max_val)
  return df[filter]

def abs_mean_value_upto_l(pd_series,L_max:int):
  idx = pd_series.index
  abs_cum_sum = 0
  num = 0
  for i in idx:
    L = int(i.split(",")[0][3:])
    if L > L_max:
      continue
    else:
      abs_cum_sum = abs_cum_sum+abs(pd_series[i])
      num = num +1
  return abs_cum_sum/num

def get_mode(name):
  return int(name.split("(")[-1].split(")")[0])
def get_radii(name):
  if name[-5]=='R':
    # R0257 -> 0257load_and_pickle
    return int(name.split('_')[-1][1:])
  else:
    return int(name.split('_')[-1])
def sort_by_power_modes(col_names):
  col_name_copy = list(col_names).copy()
  return sorted(col_name_copy, key=lambda x: int(get_mode(x)))

def add_L_mode_power(df:pd.DataFrame,L:int, ReOrIm:str):
  column_names = df.columns
  n = 0
  power = 0
  for m in range(-L,L+1):
    col_name = f'{ReOrIm}({L},{m})'
    # print(col_name)
    if col_name in column_names:
      power = power + df[col_name]*df[col_name]
      n = n + 1
  if n != 0:
    power = power/n
    df[f'pow_{ReOrIm}({L})'] = power
  return power

def add_all_L_mode_power(df:pd.DataFrame,L_max:int):
  local_df = df.copy()
  total_power_Re = 0
  total_power_Im = 0
  for l in range(0,L_max+1):
    total_power_Re = total_power_Re + add_L_mode_power(local_df,l,"Re")
    total_power_Im = total_power_Im + add_L_mode_power(local_df,l,"Im")
    local_df[f"pow_cum_Re({l})"] = total_power_Re
    local_df[f"pow_cum_Im({l})"] = total_power_Im
  return local_df


def create_power_diff_dict(power_dict:dict, pow_mode:str, variable:str, base_key:str):
  diff_dict = {"t(M)":power_dict[base_key]['t(M)']}
  y_base = power_dict[base_key][variable][pow_mode]
  y_norm = np.linalg.norm(y_base)
  for key in power_dict:
    if key == base_key:
      continue
    y = power_dict[key][variable][pow_mode]
    t = power_dict[key]['t(M)']
    y_interpolator = interp1d(t, y, kind='cubic',fill_value='extrapolate')
    y_inter = y_interpolator(diff_dict['t(M)'])
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

## Functions to compute the mismatch

In [ ]:

class SplineArray:
    def __init__(self, x, y):
        self.complex = np.iscomplexobj(y)
        if self.complex:
            y = y.view(dtype=float)
        self.splines = [Spline(x, y[:, i]) for i in range(y.shape[1])]

    def __call__(self, xprime):
        yprime = np.concatenate(
            [spl(xprime)[:, np.newaxis] for spl in self.splines], axis=1
        )
        if self.complex:
            yprime = yprime.view(dtype=complex)
        return yprime

def SquaredError(W1, W2, t1, t2, modes=None, return_h1h2_h1h1=False):
    """
    Calculate the residue of W1 and W2 between t1 and t2.
    """
    W2_spline = SplineArray(W2.t, W2.data)
    t_filter = (W1.t >= t1) & (W1.t <= t2)
    filtered_time = W1.t[t_filter]

    if modes is None:
        h1h2 = np.sum(
            sp.integrate.simpson(
                abs(W2_spline(filtered_time) - W1.data[t_filter, :]) ** 2.0,
                filtered_time,
                axis=0,
            )
        )
        h1h1 = np.sum(
            sp.integrate.simpson(
                abs(W1.data[t_filter, :]) ** 2.0, filtered_time, axis=0
            )
        )
    else:
        h1h2 = np.sum(
            sp.integrate.simpson(
                abs(W2_spline(filtered_time) - W1.data[t_filter, :])[:, modes] ** 2.0,
                filtered_time,
                axis=0,
            )
        )
        h1h1 = np.sum(
            sp.integrate.simpson(
                abs(W1.data[t_filter, :][:, modes]) ** 2.0, filtered_time, axis=0
            )
        )
    if return_h1h2_h1h1:
        return h1h2 , h1h1
    else:
        return 0.5 * h1h2 / h1h1

In [ ]:
t_interpolate = np.linspace(-1000,4000,num=5000)
data_path = Path("/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/GW_data_lev6/BondiCceR0250/red_cce.h5")
data = load_and_pickle(data_path,options = {'t_interpolate':t_interpolate})
data = load_bondi_constraints(data_path)

# Multiple ABDs

In [ ]:
cce_data = {}
# cce_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/red_cce.h5")
# cce_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/red_cce.h5")
# cce_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/red_cce.h5")
# cce_data["high_accuracy_Lev3_R0472"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0472/red_cce.h5")
# cce_data["high_accuracy_Lev4_R0472"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0472/red_cce.h5")
# cce_data["high_accuracy_Lev5_R0472"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0472/red_cce.h5")
# cce_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev4_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev5_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5/BondiCceR0257/red_cce.h5")
# cce_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/red_cce.h5")
# cce_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/red_cce.h5")
# cce_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# cce_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")

# cce_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")

# cce_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")

levs,run_sets,radius= [],[],[]
# levs = [0,1,2,3,4,5,6]
# levs = [1,2,3,4,5,6]
# levs = [1,3,6]
# levs = [0,1,2,3]
# levs = [5]
# levs = [5,6]
# levs = [6]
# run_sets = [1]
# radius = [250]
# radius = [100,150,200,250,300,350,500,700,900]
# radius = [150,200,250,300,350,500,700]
# radius = [200,250,300,350,500]
for l,s,r in itertools.product(levs,run_sets,radius):
  if s == 2 and (l == 0 or l==1):
    continue
  if l <= 3:
    if s == 1:
      cce_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5")
    # cce_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5")
  else:
    cce_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5")
    pass


# cce_data[f"6_set1_L6s3_CAMR_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_CAMR/GW_data_lev3/BondiCceR0{r}/red_cce.h5")
# cce_data[f"6_set1_L6s3_min_L_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_L/GW_data_lev3/BondiCceR0{r}/red_cce.h5")

# cce_data[f"7_constAMR_set1_L6_base_{l}_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5/")

# cce_data[f"10_4000M_CAMR_set1_L6_base_{l}_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5/")
# cce_data[f"11_4000M_CAMR_set1_L6_maxExt_{l}_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5/")

# cce_data[f"6_set1_L3s3_3"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/GW_data_lev2/BondiCceR0250/red_cce.h5")

# radius_list = []
# radius_list = ['0100','0150','0200','0250','0300','0350','0500','0700','0900','1100','1300','1400']
# radius_list = ['0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"12_set1_L3_1500_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/12_set1_L3_1500/GW_data_lev3/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100','0150','0200','0250','0300','0350','0500','0700','0900','1100','1300','1500','1700','1900']
# radius_list = ['0200','0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"12_set1_L3_2000_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/12_set1_L3_2000/GW_data_lev3/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100','0150','0200','0250','0300','0350','0500','0700','0900','1100','1300','1500','1700','1900','2100','2300']
# radius_list = ['0200','0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"12_set1_L3_2500_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/12_set1_L3_2500/GW_data_lev3/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900']
# # radius_list = ['0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"13_set1_L3_3000_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/13_set1_L3_3000/GW_data_lev3/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400']
# radius_list = ['0200','0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"13_set1_L4_1500_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/GW_data_lev4/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0400', '0500', '0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900']
# radius_list = ['0200','0250','0300','0350','0500','0700']
# for r in radius_list:
#   cce_data[f"13_set1_L4_3000_{r}"] =  Path(f"/groups/sxs/hchaudha/spec_runs/13_set1_L4_3000/GW_data_lev4/BondiCceR{r}/red_cce.h5/")

# radius_list = []
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0400', '0500', '0600', '0700', '0800', '0900']
# for r in radius_list:
#   cce_data[f"16_set1_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/16_set1_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"16_set1_L3_HP32_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"16_set1_L3_HP28_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP28/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"16_set1_L3_HP32_AF_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32_AF/GW_data_lev3/BondiCceR{r}/red_cce.h5")

# radius_list = []
# radius_list = ['0258', '0472', '0686', '0900']
# for r in radius_list:
#   cce_data[f"17_main_9_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")

# radius_list = []
# radius_list = ['0258', '0469', '0679', '0890']
# for r in radius_list:
#   cce_data[f"17_set_main_q3_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")

# radius_list = []
# radius_list = ['0199', '0353', '0506', '0660']
# for r in radius_list:
#   cce_data[f"17_set_main_q3_15_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set_main_q3_15_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")

# radius_list = []
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0500', '0700', '0900']
# radius_list = [ '0300', '0250', '0350','0200']
# radius_list = [ '0350']
# for r in radius_list:
#   cce_data[f"17_set1_q3_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"17_set3_q3_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set3_q3_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"17_set1_9_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"17_set3_9_18_L3_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/17_set3_9_18_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"22_set1_L1_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_set1_L1_long/GW_data_lev1/BondiCceR{r}/red_cce.h5")
#   cce_data[f"22_set1_L3_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"22_L3_AC_L3_no_res_C_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_no_res_C/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"22_L3_AC_L3_res_10_C_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_res_10_C/GW_data_lev3/BondiCceR{r}/red_cce.h5")

#   cce_data[f"L1_AC_L2_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L2/GW_data_lev2/BondiCceR{r}/red_cce.h5")
#   cce_data[f"L1_AC_L3_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L3/GW_data_lev3/BondiCceR{r}/red_cce.h5")
#   cce_data[f"L3_AC_L1_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L1/GW_data_lev1/BondiCceR{r}/red_cce.h5")
#   cce_data[f"L3_AC_L2_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L2/GW_data_lev2/BondiCceR{r}/red_cce.h5")
#   cce_data[f"L3_AC_L4_long_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L4/GW_data_lev4/BondiCceR{r}/red_cce.h5")

# levs,radius,start = [],[],[]
# levs = [1,3]
# levs = [3]
# radius = ['0250', '0350']
# radius = ['0350']
# start = [3000,5000,7000,8000]
# start = [3000,7000]
# for l,r,s in itertools.product(levs,radius,start):
#   cce_data[f"L{l}_S{s}_r{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_cce_test/L{l}/start_{s}/BondiCceR{r}/red_cce.h5")

# levs,radius,start = [],[],[]
# levs = ['_IC','_NIR','_ZNS','']
# levs = ['_NIR','']
# radius = ['0250', '0350']
# radius = ['0350']
# start = [0,500,1000,3000,7000]
# start = [3000,7000]
# for l,r,s in itertools.product(levs,radius,start):
#   cce_data[f"L3{l}_S{s}_r{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_cce_test/L3/start_{s}{l}/BondiCceR{r}/red_cce.h5")

# radius = ['0020','0035','0050','0075','0100','0150','0200','0250','0300','0400','0500','0600','0800','1000','1500','2000','2500',]
# radius = ['0020','0050','0100','0200','0500','1000','1500','2000','2500',]
# for r in radius:
#   cce_data[f"14_NIR_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14_NIR/BondiCceR{r}/red_cce.h5")


# radius = ['0012', '0050', '0112', '0200', '0312', '0450', '0612', '0800', '1012', '1250', '1512', '1800', '2112', '2450', '2812', '3200', '3612', '4050', '4512', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0015', '0020', '0030', '0075']
# radius = sorted(['0050', '0112', '0200', '0612', '0800', '1012',  '1800',  '3200', '3612', '4050', '4512', '0006', '0020', '0075'])
# radius = sorted(['0050', '0112', '0200', '0612', '0800', '1012',  '0006', '0010', '0015', '0020', '0030', '0075','1800',  '3200','4512'])
# radius = ['0006']
# for r in radius:
#   cce_data[f"ex_rad_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh_CCE/runs/radius_dependence/ex_rad_{r}/red_cce.h5")

# radius = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000 ]
# radius = [0, 100, 200, 300, 400, 500, 600]
# for r in radius[::]:
#   cce_data[f"CF_350_start_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/22_cce_test/L3_new_executable/runs/ConformalFactor_start_{r}/red_cce.h5")

# for i in Path("/groups/sxs/hchaudha/spec_runs/22_cce_test/L3_merged_data/runs").glob("*/red_cce.h5"):
#     cce_data[i.parent.stem] = i
# for i in Path("/groups/sxs/hchaudha/spec_runs/22_cce_test/L3_merged_data/runs").glob("Delta*/red_cce.h5"):
#     cce_data[i.parent.stem] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/22_cce_test/runs/CF_0").glob(
#     "Delta*/red_cce.h5"
# ):
#     cce_data[i.parent.stem] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/GW_data_lev3_start_6000").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     cce_data["set1_L3_long_ST_6000M_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/GW_data_lev3_start_4000").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     cce_data["set1_L3_long_ST_4000M_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/GW_data_lev3_start_2000").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     cce_data["set1_L3_long_ST_2000M_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/32_RM_set1_L3/GW_data_lev3").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     cce_data["32_RM_set1_L3_"+i.parent.stem[-4:]] = i

def include_radii(name, min,max):
    radius = int(name[-4:])
    if radius < min or radius > max:
        return False
    else:
        return True

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_250/GW_data_lev3").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     if not include_radii(i.parent.stem, 200, 400):
#         continue
#     cce_data["32_RM_set1_L1s3_cdg1_250_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_100/GW_data_lev3").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     if not include_radii(i.parent.stem, 200, 400):
#         continue
#     cce_data["32_RM_set1_L1s3_cdg1_100_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_10/GW_data_lev3").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     if not include_radii(i.parent.stem, 200, 400):
#         continue
#     cce_data["32_RM_set1_L1s3_cdg1_10_"+i.parent.stem[-4:]] = i

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3/GW_data_lev3").glob(
#     "BondiCceR*/red_cce.h5"
# ):
#     if not include_radii(i.parent.stem, 200, 400):
#         continue
#     cce_data["32_RM_set1_L1s3_"+i.parent.stem[-4:]] = i

for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin18/GW_data_lev3").glob(
    "BondiCceR*/red_cce.h5"
):
    if not include_radii(i.parent.stem, 200, 400):
        continue
    cce_data["set1_L3_Lmin18_"+i.parent.stem[-4:]] = i

for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin20_Rn2/GW_data_lev3").glob(
    "BondiCceR*/red_cce.h5"
):
    if not include_radii(i.parent.stem, 200, 400):
        continue
    cce_data["set1_L3_Lmin20_Rn2_"+i.parent.stem[-4:]] = i

for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn1/GW_data_lev3").glob(
    "BondiCceR*/red_cce.h5"
):
    if not include_radii(i.parent.stem, 200, 400):
        continue
    cce_data["set1_L3_Rn1_"+i.parent.stem[-4:]] = i

for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn2/GW_data_lev3").glob(
    "BondiCceR*/red_cce.h5"
):
    if not include_radii(i.parent.stem, 200, 400):
        continue
    cce_data["set1_L3_Rn2_"+i.parent.stem[-4:]] = i

# cce_data['6_12_250'] = Path("/resnick/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/GW_data_lev6_12/BondiCceR0250/red_cce.h5")
# cce_data['6_10_250'] = Path("/resnick/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/GW_data_lev6_10/BondiCceR0250/red_cce.h5")
# cce_data['BondiCceR0334'] = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/Lev4_061CCE/BondiCceR0334/red_cce.h5")
# cce_data['BondiCceR0586'] = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/Lev4_061CCE/BondiCceR0586/red_cce.h5")

cce_data = dict(sorted(cce_data.items()))


fail_flag = False
for key in cce_data:
    if not cce_data[key].exists():
        fail_flag = True
        print(f"{cce_data[key]} does not exist!")
    if fail_flag:
        raise Exception("Some paths do not exist!")

print(cce_data.keys())

In [ ]:
t_interpolate = np.linspace(-1000,20000,num=2000)
# t_interpolate = np.linspace(-1000,4000,num=10000)

abd_data = {}
failed_keys=  {}
for key in cce_data:
  try:
    # abd_data[key] = load_and_pickle(cce_data[key])
    # abd_data[key] = load_and_pickle(cce_data[key], reload_data=True)
    # abd_data[key] = load_and_pickle(cce_data[key],options = {'t_interpolate':t_interpolate}, reload_data=True)
    abd_data[key] = load_and_pickle(cce_data[key],options = {'t_interpolate':t_interpolate})
    abd_data[key] = load_bondi_constraints(cce_data[key])
  except Exception as e:
    failed_keys[key] = str(e)
    print(f"Failed to load and pickle data for key {key}: {e}")
    continue

print(abd_data.keys())

In [ ]:
bondi_norms_to_plot = [0, 1, 2, 3, 4, 5]
# bondi_norms_to_plot = [0,1,3,4,2]
# bondi_norms_to_plot = [2]

t_min = -10000
# t_min = 700
# t_min = 2500
# t_min = 7500
# t_min = 4000 - 250
t_max = 100000
# t_max = 7751.5
# t_max = 1200
# t_max = 3600
t_max = 4000
# t_max = 7000

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }


def get_radius(run_name):
    radius_part = run_name.split("_")[-1]
    a = "".join([i for i in radius_part if i.isdigit()])
    return int(a)


min_radius = 0
min_radius = 200
# min_radius = 310
max_radius = 1000
max_radius = 360
# max_radius = 460

line_styles = cycle(["-", "--", "-.", ":"])
y_list = []
for num_key, key in enumerate(abd_data):
    # if "6" not in key:
    #     continue

    line_style = next(line_styles)
    plt.gca().set_prop_cycle(None)
    #   if num_key%6 == 0:
    #     plt.gca().set_prop_cycle(None)
    violation_dict = abd_data[key]["bondi_violation_norms"]
    #   radius = get_radius(key)
    #   if radius > max_radius or radius < min_radius:
    #     continue
    #   if "_12_" not  in key:
    #     continue
    #   if "cdg1" not in key:
    #     continue
    # if "_main_q3_18" not in key:
    #   continue
    #   if "L6s6" not in key:
    #     continue
    #   if "3000" not in key:
    #     continue
    #   if not re.search(r'L6s[1,2,3,4,5,6]_200',key):
    #     continue
    #   if not re.search(r'[\_,0]250$',key):
    #     continue
    # if not re.search(r'[\_,0][2-9]??$',key):
    #   continue

    t_arr = abd_data[key]["abd"].t
    trimmed_indices = (t_arr > t_min) & (t_arr < t_max)
    t_arr = t_arr[trimmed_indices]

    for i in bondi_norms_to_plot:
        if labels is None:
            plt.semilogy(t_arr, violation_dict[i][trimmed_indices], line_style, label=f"{key}_{i}")
        #   plt.plot(t_arr,violation_dict[i][trimmed_indices],line_style, label=f"{key}")
        # plt.semilogy(t_arr,violation_dict[i][trimmed_indices])
        else:
            plt.semilogy(
                t_arr, violation_dict[i][trimmed_indices], line_style, label=f"{labels[key]}_{i}"
            )

    plt.xlabel("t(M)")
    plt.ylabel(f"bondi violations {bondi_norms_to_plot}")
    plt.legend()
    plt.tight_layout()
#   plt.show()
# plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
# plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


In [ ]:
last_val = {}
r = []
val = []
for key in abd_data.keys():
    r.append(int(key.split("_")[-1]))
    val.append(abd_data[key]['bondi_violation_norms'][2][-1])


In [ ]:
x = abd_data['14_NIR_0006']['abd'].t
y = abd_data['14_NIR_0006']['bondi_violation_norms'][2]
x = np.log10(x)
y = np.log10(y)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fit a line y = m*x + b
coeffs = np.polyfit(x, y, deg=1)
m, b = coeffs

# Predict y values
y_fit = m * x + b

# Print results
print(f"Slope: {m}, Intercept: {b}")

# Plotting
plt.scatter(x, y, label='Data')
plt.plot(x, y_fit, color='red', label=f'Fit: y = {m:.2f}x + {b:.2f}')
plt.xlabel('log10(Extraction Radius)')
plt.ylabel('log10(Bondi violation 2)') 
plt.legend()
plt.show()


### Other bondi stuff

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [2]

t_min = -10000
# t_min = 1200
# t_min = 2500
# t_min = 7500
# t_min = 4000 - 250
t_max = 100000
# t_max = 7751.5
# t_max = 1200
# t_max = 3600
# t_max = 4000
# t_max = 7000

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

def get_radius(run_name):
  radius_part = run_name.split("_")[-1]
  a = "".join([i for i in radius_part if i.isdigit()])
  return int(a)

min_radius = 0
# min_radius = 260
max_radius = 1000
# max_radius = 360

y_list = []
for key in list(abd_data.keys())[::-1]:
#   violation_dict = abd_data[key]['bondi_violation_norms']
#   radius = get_radius(key)
#   if radius > max_radius or radius < min_radius:
#     continue
  # if "Factor" in key:
  #   continue
#   if "high" in key:
#     continue
  # if "_main_q3_18" not in key:
  #   continue
#   if "L6" not in key:
#     continue
#   if "s6" not in key:
#     continue
#   if "3000" not in key:
#     continue
#   if not re.search(r'L6s[1,2,3,4,5,6]_200',key):
#     continue
#   if not re.search(r'[\_,0]250$',key):
#     continue
  # if not re.search(r'[\_,0][2-9]??$',key):
  #   continue
  print(key)
  violation_dict = abd_data[key]['abd'].bondi_rest_mass()
#   violation_dict = abd_data[key]['abd'].bondi_angular_momentum()

  t_arr = abd_data[key]["abd"].t
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  print(key)
  violation_dict = abd_data[key]['abd'].bondi_rest_mass() - 1.0
  plt.plot(t_arr,violation_dict[trimmed_indices],label=f"{key}")


#   violation_dict = abd_data[key]['abd'].bondi_angular_momentum()
#   plt.plot(t_arr,violation_dict[:,0][trimmed_indices],label=f"{key}_0")
#   plt.plot(t_arr,violation_dict[:,1][trimmed_indices],label=f"{key}_1")
#   plt.plot(t_arr,violation_dict[:,2][trimmed_indices],label=f"{key}_2")


  plt.xlabel('t(M)')
  plt.ylabel("bond_rest_mass - 1.0")
#   plt.ylabel(f"bondi violations ")
  plt.legend()
  plt.tight_layout()
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")

In [ ]:
plt.ylabel("bond_rest_mass - 1.0")

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [2]

t_min = -10000
t_min = 2000
t_min = 2500
# t_min = 7500
# t_min = 4000 - 250
t_max = 10000
# t_max = 7751.5
# t_max = 1200
t_max = 3600
# t_max = 4000
# t_max = 7000

labels = None
labels={
#  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
 "6_set1_L6s1_250" : "New Level 1",
 "6_set1_L6s2_250" : "New Level 2",
 "6_set1_L6s3_250" : "New Level 3",
 "6_set1_L6s4_250" : "New Level 4",
 "6_set1_L6s5_250" : "New Level 5",
"high_accuracy_Lev1_R0257": "Old Level 1",
"high_accuracy_Lev2_R0257": "Old Level 2",
"high_accuracy_Lev3_R0258": "Old Level 3",
"high_accuracy_Lev4_R0258": "Old Level 4",
"high_accuracy_Lev5_R0258": "Old Level 5",
}

def get_radius(run_name):
  radius_part = run_name.split("_")[-1]
  a = "".join([i for i in radius_part if i.isdigit()])
  return int(a)

min_radius = 0
# min_radius = 260
max_radius = 1000
max_radius = 360

y_list = []

with plt.style.context('ggplot'):
    plt.rcParams["figure.figsize"] = (8,4)
    # plt.rcParams["figure.figsize"] = (10,8)
    fig,ax = plt.subplots(1, 2)


    for key in ['high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257', 'high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258']:
        violation_dict = abd_data[key]['bondi_violation_norms']
        t_arr = abd_data[key]["abd"].t
        trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
        t_arr = t_arr[trimmed_indices]

        ax[0].semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{labels[key]}")
        ax[0].set_xlabel('t(M)')
        ax[0].legend(loc='upper right')
        ax[0].set_ylim(5e-8,3e-6)

    for key in ['6_set1_L6s1_250', '6_set1_L6s2_250', '6_set1_L6s3_250', '6_set1_L6s4_250', '6_set1_L6s5_250']:
        violation_dict = abd_data[key]['bondi_violation_norms']
        t_arr = abd_data[key]["abd"].t
        trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
        t_arr = t_arr[trimmed_indices]

        ax[1].semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{labels[key]}")
        ax[1].legend(loc='upper right')
        ax[1].set_xlabel('t(M)')
        ax[1].set_ylim(5e-8,3e-6)

    fig.suptitle("Bondi constraint violations in CCE waveforms")
    plt.legend()
    plt.tight_layout()
    plt.savefig(Path("/groups/sxs/hchaudha/scripts/report/figures")/f"Bondi constraints.png", dpi=600)

In [ ]:


one_over_x = y_list[0]*1e2/np.array(radius)
one_over_x2 = y_list[0]*1e4/np.array(radius)**2

# Define the fitting function
def fitting_func(r, a, b):
    return a / r**b
popt, pcov = curve_fit(fitting_func, radius, y_list,p0=[y_list[0],2])
a_fit, b_fit = popt

# Create a smooth curve for the fitted function
fit_vals = fitting_func(radius, a_fit, b_fit)

plt.plot(radius,y_list, label=f"power in mode ",marker='x')
# plt.plot(radius,one_over_x, label="1/x",marker='x')
# plt.plot(radius,one_over_x2, label="1/(x*x)",marker='x')
plt.plot(radius,fit_vals,label=f'Fit: a/r^b, a={a_fit:.2e}, b={b_fit:.2f}', linestyle='--')
plt.yscale('log')
plt.legend()
plt.xlabel('r(M)')
# plt.ylabel('power in modes')
# plt.title(f"L={L_list}")

### Plot cce waveform diff

In [ ]:
def get_radius(run_name):
  radius_part = run_name.split("_")[-1]
  a = "".join([i for i in radius_part if i.isdigit()])
  return int(a)

for i in abd_data.keys():
  print(i)

In [ ]:
modes_to_plot = [(3,2)]
modes_to_plot = [(2,2)]

# base_key = 'high_accuracy_Lev5_R0258'
# base_key = 'high_accuracy_Lev2_R0257'
# base_key = '6_set1_L6s4_250'
base_key = '22_set1_L3_long_0300'
# base_key = '22_set1_L3_long_0250'
# base_key = '12_set1_L3_2500_0350'
# base_key = '6_set1_L3s3_250'
# base_key = '6_set1_L6s6_250'
radius = get_radius(base_key)

plot_vals = ["_diff","_absdiff","_rel_diff","_rel_absdiff"]
plot_vals = ["_absdiff","_rel_absdiff"]
# plot_vals = ["_rel_absdiff"]

t_min = 0
# t_min = 2000
# t_min = 2500
# t_min = 1190
# t_min = 4000
t_max = 10000
# t_max = 7751.5
# t_max = 7000
# t_max = 6000
# t_max = 3500
t_interpolate = np.linspace(t_min,t_max,num=5000)

def filter(key):
  min_radius = 0
  min_radius = 200
  max_radius = 1000
#   max_radius = 550

  radius = get_radius(key)
  if radius > max_radius or radius < min_radius:
    return True
  if key == base_key:
    return True
  if 'L3_long_0250' not in key:
    return True
  # if not re.search(r"Lev[3,4,5]",key):
  #   return True
#   if not re.search(r"s[0,1,2]",key):
#     return True
  # if 's3' in key:
  #   return True

num_colors_required = len(modes_to_plot)*(len(abd_data)-1)

num_colors_required = 0
for mode,key in itertools.product(modes_to_plot,abd_data):
  if filter(key):
    continue
  num_colors_required = num_colors_required + 1

if num_colors_required <=10:
  cmap = plt.get_cmap('tab10')
  color = [cmap(i) for i in range(num_colors_required )]
elif num_colors_required <=20:
  cmap = plt.get_cmap('tab20')
  color = [cmap(i) for i in range(num_colors_required )]
else:
  color=plt.cm.hsv(np.linspace(0, 1, num_colors_required))
  # color=plt.cm.gist_rainbow(np.linspace(0, 1, num_colors_required))

i=-1
plt.close()
for mode,key in itertools.product(modes_to_plot,abd_data):
  l,m = mode
  if filter(key):
    continue

  diff_dict = create_diff_dict_cce(abd_data,l=l,m=m,base_key=base_key,t_interpolate=t_interpolate)
  t_arr = diff_dict["t"]

  i = i+1
  for plot_val in plot_vals:
    match plot_val:
      case "_rel_absdiff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="-",color=color[i])
      case "_absdiff":
        # plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle=":",color=color[i])
        plt.semilogy(t_arr,diff_dict[key+plot_val],linestyle=":",color=color[i])
      case "_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="--",color=color[i])
      case "_rel_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="-.",color=color[i])

    plt.xlabel('t')
    plt.ylabel("diff")
    plt.legend()
    plt.tight_layout()
    plt.title(f"CCE waveform diff {modes_to_plot} {base_key=}")
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
plt.show()


# CCE frame fixing

In [ ]:
from copy import deepcopy

wave1 = deepcopy(abd_data['22_set1_L3_long_0300']['abd'])
wave2 = deepcopy(abd_data['22_set1_L3_long_0350']['abd'])
wave2 = deepcopy(abd_data['22_L3_AC_L3_res_10_C_0300']['abd'])

In [ ]:
abd_prime1,transformations1,err1 = wave1.map_to_superrest_frame(t_0=9000)
abd_prime2,transformations2,err2 = wave2.map_to_superrest_frame(t_0=9000)

In [ ]:
l,m = 2,2

h1 = abd_prime1.h
y1 = h1.data[:, lm(2,2, h1.ell_min)]

h2 = abd_prime2.h
y2 = h2.data[:, lm(2,2, h2.ell_min)]


In [ ]:
l,m = 2,2

h1 = wave1.h
y1 = h1.data[:, lm(2,2, h1.ell_min)]

h2 = wave2.h
y2 = h2.data[:, lm(2,2, h2.ell_min)]

In [ ]:
plt.plot(h1.t,np.abs(y1), label='y1')
plt.plot(h2.t,np.abs(y2), label='y2')
plt.legend()

In [ ]:

plt.plot(h1.t,y1-y2, label='y1-y2')
plt.legend()

# Finite radius waveform comparison

In [ ]:
def extract_radii(h5_file_path:Path):
  radii = set()
  with h5py.File(h5_file_path,'r') as f:
    names = []
    f.visit(names.append)
  for name in names:
    if "Version" in name:
      continue
    radii.add(name[1:5])
  radii = list(radii)
  radii.sort()
  return radii

In [ ]:
GW_data_path= {}
# GW_data_path["master_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data")
# GW_data_path["master_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1")
# GW_data_path["master_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2")
# GW_data_path["master_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3")
# GW_data_path["master_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4")
# GW_data_path["master_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5")

# GW_data_path["high_accuracy_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev0_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev1_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev2_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev45"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev45_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev55"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev55_AC/Run/GW2/")

# GW_data_path["20_zero_spin_AMR_L5_10000M"] = Path("/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14/rh_FiniteRadii_CodeUnits")

levs,run_sets,radius= [],[],[]
levs = [1,2,3,4,5,6]
# levs = [0,1,2,3]
# levs = [4,5]
# levs = [0,2,3,4,5]
# levs = [0]
run_sets = [1]
# run_sets = [1]

for l,s in itertools.product(levs,run_sets):
  if s == 2 and (l == 0 or l==1):
    continue
#   if l <= 3:
#     GW_data_path[f"6_set{s}_L3s{l}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/rh_FiniteRadii_CodeUnits")
#   else:
  GW_data_path[f"6_set{s}_L6s{l}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/rh_FiniteRadii_CodeUnits")


# levs = [1,2,3,4,5,6]
# for l in levs:
#   GW_data_path[f"main_s{l}"] = Path(f"/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/GW_data_lev{l}/rh_FiniteRadii_CodeUnits/")

file = "rh_FiniteRadii_CodeUnits.h5"
for key in GW_data_path:
  GW_data_path[key] = GW_data_path[key]/file
  if not GW_data_path[key].exists():
    raise Exception(f"{key}, {GW_data_path[key]} does not exists!")
radii_list = extract_radii(list(GW_data_path.values())[0])
print(radii_list)
print(GW_data_path.keys())

In [ ]:
GW_data = {}
# for radius in radii_list:
for radius in [radii_list[9]]:
# for radius in list(np.array(radii_list)[[0,4,8,12,15,17,19,21,23]]):
# for radius in list(np.array(radii_list)[[0,4,12,19,23]]):
    for key in GW_data_path:
        key_name = key + "_" + radius
        GW_data[key_name] = scri.SpEC.read_from_h5(
            f"{GW_data_path[key]}/R{radius}.dir",
            dataType=scri.h,
            frameType=scri.Inertial,
            r_is_scaled_out=False,
            m_is_scaled_out=False,
        )
        print(f"{key_name} loaded!")

print(GW_data.keys())


In [ ]:
from collections import defaultdict
def indices_of_a_mode(L, ell_min):
    if L is None:
        return None
    indices = []
    if L < ell_min:
        return indices
    for m in range(-L, L + 1):
        indices.append(lm(L, m, ell_min))
    return indices

def have_same_extraction_radius(key1, key2):
    rad1 = key1.split("_")[-1]
    rad2 = key2.split("_")[-1]
    if rad1 == rad2:
        return True
    else:
        return False


def pair_mismatch(GW_data, t1, t2, modes=None, pair_alg="all",split_by_radius=False):
    mismatch = {}

    # This is to deal with the situation where we have multiple levels with multiple extraction radii.
    dict_by_radius = defaultdict(dict)
    for key, val in GW_data.items():
        suffix = key.split("_")[-1]
        dict_by_radius[suffix][key] = val

    for suffix, data in dict_by_radius.items():
        keys = sorted(list(data.keys()))
        print(data.keys())
        if pair_alg == "alternate":
            for i, j in zip(keys[:-1], keys[1:]):
                mismatch[i + "@" + j] = SquaredError(
                    data[i], data[j], modes=modes, t1=t1, t2=t2
                )
        elif pair_alg == "wrt_highest":
            j = keys[-1]
            for i in keys[:-1]:
                mismatch[i + "@" + j] = SquaredError(
                    data[i], data[j], modes=modes, t1=t1, t2=t2
                )
        elif pair_alg == "all":
            for i, j in itertools.combinations(keys, 2):
                mismatch[i + "@" + j] = SquaredError(
                    data[i], data[j], modes=modes, t1=t1, t2=t2
                )
        else:
            raise ValueError(
                f"Unknown pair algorithm: {pair_alg}, must be one of 'alternate', 'wrt_highest', 'all'"
            )

    if split_by_radius:
        mismatch_by_radius = defaultdict(dict)
        for key, val in mismatch.items():
            radius = key.split("@")[0].split("_")[-1]
            mismatch_by_radius[radius][key] = val
        return mismatch_by_radius
    else:
        return mismatch


In [ ]:
Lmode = None
# Lmode = 8
mode_indices = indices_of_a_mode(Lmode, GW_data[list(GW_data.keys())[0]].ell_min)

# mismatch_dict = pair_mismatch(GW_data, t1=1200, t2=4000, pair_alg="alternate",modes=mode_indices)
mismatch_dict = pair_mismatch(GW_data, t1=1200, t2=4000, pair_alg="alternate", modes=mode_indices)
# mismatch_dict = pair_mismatch(GW_data, t1=1200, t2=4000, pair_alg="all",modes=mode_indices)
mismatch_dict


In [ ]:

fig,ax = plt.subplots(figsize=(8, 6))

filter_dict = {
    's1,s2': {},
    's2,s3': {},
    's3,s4': {},
    's4,s5': {},
    's5,s6': {},
}
for key, val in mismatch_dict.items():
    for s in filter_dict.keys():
        matching_keys = True
        for i in s.split(","):
            if i not in key:
                matching_keys = False
        if matching_keys:
            filter_dict[s][key] = val

for s, mismatch_dict_local in filter_dict.items():
    radius_arr = [int(key.split("_")[-1]) for key in mismatch_dict_local.keys()]
    val_arr = [i for i in mismatch_dict_local.values()]
    ax.scatter(radius_arr, val_arr, label=s)

# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Extraction Radius (M)")
if Lmode is not None:
    ax.set_ylabel(f"Mismatch for L={Lmode}")
    ax.set_title(f"Mismatch for L={Lmode} vs Extraction Radius")
else:
    ax.set_ylabel("Mismatch")
    ax.set_title("Mismatch vs Extraction Radius")
plt.legend(loc='upper right')
plt.tight_layout()

In [ ]:
radius_arr = [int(key.split("_")[-1]) for key in mismatch_dict.keys()]
val_arr = [i for i in mismatch_dict.values()]

fig,ax = plt.subplots(figsize=(8, 6))

ax.scatter(radius_arr, val_arr)
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Extraction Radius (M)")
if Lmode is not None:
    ax.set_ylabel(f"Mismatch for L={Lmode}")
    ax.set_title(f"Mismatch for L={Lmode} vs Extraction Radius")
else:
    ax.set_ylabel("Mismatch")
    ax.set_title("Mismatch vs Extraction Radius")
plt.tight_layout()

#### Variation by modes

Load a single extraction radii with multiple levs

In [ ]:
mismatch_dict = {}
for Lmode in ["all", 2, 3, 4, 5, 6, 7, 8]:
    if Lmode == "all":
        mode_indices = None
    else:
        mode_indices = indices_of_a_mode(
            Lmode, GW_data[list(GW_data.keys())[0]].ell_min
        )
    # mismatch_dict[str(Lmode)] = pair_mismatch(GW_data, t1=1200, t2=4000, pair_alg="alternate",modes=mode_indices)
    mismatch_dict[str(Lmode)] = pair_mismatch(
        GW_data, t1=1200, t2=4000, pair_alg="alternate", modes=mode_indices
    )
    # mismatch_dict[str(Lmode)] = pair_mismatch(GW_data, t1=1200, t2=4000, pair_alg="all",modes=mode_indices)
mismatch_dict


In [ ]:

fig,ax = plt.subplots(figsize=(8, 6))

filter_dict = {
    's1,s2': {},
    's2,s3': {},
    's3,s4': {},
    's4,s5': {},
    's5,s6': {},
}

for key, val in mismatch_dict.items():
    for key1, val1 in val.items():
        for s in filter_dict.keys():
            matching_keys = True
            for i in s.split(","):
                if i not in key1:
                    matching_keys = False
            if matching_keys:
                filter_dict[s][key] = val1

for s, mismatch_dict_local in filter_dict.items():

    x_labels = mismatch_dict_local.keys()
    val_arr = [i for i in mismatch_dict_local.values()]
    ax.scatter(range(len(x_labels)), val_arr, label=s)

# ax.set_xscale('log')

ax.set_xticks(range(len(x_labels)))
ax.set_xticklabels(x_labels, ha='right')

ax.set_yscale('log')
ax.set_xlabel("Modes")
ax.set_ylabel("Mismatch")
ax.set_title(list(GW_data.keys())[-1])
ax.set_ylim(1e-11, 1e-1)
plt.legend(loc='upper left')
plt.tight_layout()

### Older stuff

In [ ]:
l, m = 2,2
t_max = 3500
t_interpolate = np.linspace(1200+int(radius),t_max+int(radius),num=5000)

def np_abs_func(x):
  return np.abs(np.real(x))
  # return np.abs(x)

for key in GW_data:
#   if 'L6' not in key:
#     continue
#   if not re.search(r'Lev[3,4,5]',key):
#     continue

  h = GW_data[key].interpolate(t_interpolate)
  shited_time = h.t - int(radius)
  diff = h.data[:, lm(l,m,h.ell_min)] 
  diff = np_abs_func(diff)
  plt.plot(shited_time,diff,label=f'{key}_{l=}_{m=}')
  # plt.plot(shited_time,diff_normalize,label=f'{key}_rel_{l=}_{m=}')
  # plt.plot(shited_time,base_h.data[:, lm(l,m,h.ell_min)],label=f'{key}_rel2_{l=}_{m=}')
  # plt.plot(shited_time,np_abs_func(base_h.data[:, lm(l,m,h.ell_min)]),label=f'{key}_rel2_{l=}_{m=}')
  # plt.semilogy(h.t,h.energy_flux(),label=f'{key}')

plt.title(f"diff:{base_key}  {l=},{m=}, {radius=}")
plt.xlabel('t')
plt.ylabel("diff")
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/del/LevVariations")/f"L05_finite_radius_{base_key}_{l=}_{m=}_{radius}_{t_max=}.png")

## Diff

In [ ]:

l, m = 2,2
t_max = 3500
t_interpolate = np.linspace(1200+int(radius),t_max+int(radius),num=5000)

base_key = 'master_Lev5'
# base_key = 'master_Lev2'
base_key = 'high_accuracy_Lev5'
# base_key = 'high_accuracy_Lev2'
# base_key = '6_set1_L6s6'
# base_key = '6_set3_L6s5'
# base_key = '6_set1_L3s3'
# base_key = '6_set2_L3s2'
base_h = GW_data[base_key].interpolate(t_interpolate)

def np_abs_func(x):
  return np.abs(np.real(x))
  # return np.abs(x)

for key in GW_data:
  if 'L6' not in key:
    continue
#   if not re.search(r'Lev[3,4,5]',key):
#     continue
  if key == base_key:
    continue
  h = GW_data[key].interpolate(t_interpolate)
  shited_time = h.t - int(radius)
  diff = h.data[:, lm(l,m,h.ell_min)] - base_h.data[:, lm(l,m,h.ell_min)]
  diff = np_abs_func(diff)
  diff_normalize = diff/np_abs_func(base_h.data[:, lm(l,m,h.ell_min)])
  plt.plot(shited_time,diff,label=f'{key}_{l=}_{m=}')
  # plt.plot(shited_time,diff_normalize,label=f'{key}_rel_{l=}_{m=}')
  # plt.plot(shited_time,base_h.data[:, lm(l,m,h.ell_min)],label=f'{key}_rel2_{l=}_{m=}')
  # plt.plot(shited_time,np_abs_func(base_h.data[:, lm(l,m,h.ell_min)]),label=f'{key}_rel2_{l=}_{m=}')
  # plt.semilogy(h.t,h.energy_flux(),label=f'{key}')

plt.title(f"diff:{base_key}  {l=},{m=}, {radius=}")
plt.xlabel('t')
plt.ylabel("diff")
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/del/LevVariations")/f"L05_finite_radius_{base_key}_{l=}_{m=}_{radius}_{t_max=}.png")

# World tube data

In [ ]:
WT_data= {}
# WT_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/BondiCceR0258.h5")
# WT_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev1_R0257"] = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev2_R0257"] = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev3_R0257"] = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev3/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev4_R0257"] = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev4/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev5_R0257"] = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev5/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/BondiCceR0686.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/BondiCceR0100.h5")
# WT_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# WT_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")

# WT_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")

levs,run_sets,radius= [],[],[]
# levs = [1,2,3,4,5,6]
levs = [2,3,4,5,6]
# levs = [5,4]
# levs = [0,1,2,3]
# levs = [6]
# run_sets = [1,2,3]
# run_sets = [1,3]
run_sets = [1]
radius = [250]
# radius = [100,150,200,250,300,350,500,700,900]
# radius = [100,250,900]

for l,s,r in itertools.product(levs,run_sets,radius):
  # if s == 2 and (l == 0 or l==1):
  #   continue
  # if l <= 3:
  #   WT_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")
  # else:
  #   WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")

  WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")

# for i in Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/Lev4_061CCE/").glob("*.h5"):
#     WT_data[i.name.split(".")[0]] = i

fail_flag=False
for key in WT_data:
  if not WT_data[key].exists():
    fail_flag = True
    print(f"{WT_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
# ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
WT_data_dict = {}
for key in WT_data:
  WT_data_dict[key] = WT_to_pandas(WT_data[key],keys_to_load=['J.dat'])
  print(f"{key} loaded!")

print(WT_data_dict.keys())

In [ ]:
def LMPair(L, pre="Re"):
    if pre in ["Re", "Im"]:
        return [f"{pre}({L},{m})" for m in range(-L, L + 1)]
    elif pre == "ReIm":
        return [f"Re({L},{m})" for m in range(-L, L + 1)] + [
            f"Im({L},{m})" for m in range(-L, L + 1)
        ]


In [ ]:
L = 2


def power_in_mode(L, data, pre="Re"):
    # data = WT_data_dict["6_set1_L6s0_250"]['J.dat']
    all_cols = LMPair(L, pre=pre)
    cols_present = []
    for col in all_cols:
        if col in data.columns:
            cols_present.append(col)
    if len(cols_present) == 0:
        raise Exception(f"None of the columns {all_cols} are present in the data")

    num_m = len(cols_present)
    return np.linalg.norm(data[cols_present], axis=1) / (num_m)


def power_in_mode(L, data, pre="Re"):
    # Get list of possible column names
    all_cols = LMPair(L, pre=pre)
    
    # Find which columns are present in data
    cols_present = [col for col in all_cols if col in data.columns] if isinstance(data, pd.DataFrame) else [col for col in all_cols if col in data.index]
    
    if len(cols_present) == 0:
        raise Exception(f"None of the columns {all_cols} are present in the data")

    num_m = len(cols_present)
    
    if isinstance(data, pd.DataFrame):
        # DataFrame case: multiple rows
        result = np.linalg.norm(data[cols_present], axis=1) / num_m
    elif isinstance(data, pd.Series):
        # Series case: single row
        result = np.linalg.norm(data[cols_present].values) / num_m
    else:
        raise TypeError("data must be a pandas DataFrame or Series")
    
    return result


In [ ]:
print(WT_data_dict.keys())
print(WT_data_dict[list(WT_data_dict.keys())[0]].keys())

In [ ]:
t_val = 3500
key = "master_Lev5_R0257"
key = "6_set1_L6s6_250"
# key = "BondiCceR0334"
data = WT_data_dict[key]

for var in ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']:
    data = WT_data_dict[key][var]
    t = data["t(M)"]
    closest_index = (t - t_val).abs().idxmin()
    L_val = []
    pow_in_L = []
    for L in range(16):
        L_val.append(L)
        pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

    # plt.scatter(L_val, pow_in_L, label=var)
    plt.plot(L_val, pow_in_L, label=var)
plt.legend()
plt.xlabel('L')
plt.ylabel('Power in mode')
plt.title(f"{key} at {t_val=}M")
plt.yscale('log')

In [ ]:
t_val = 3500
var = 'J.dat'
for key in WT_data_dict:
    data = WT_data_dict[key][var]
    t = data["t(M)"]
    closest_index = (t - t_val).abs().idxmin()
    L_val = []
    pow_in_L = []
    for L in range(16):
        L_val.append(L)
        pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

    # plt.scatter(L_val, pow_in_L, label=var)
    plt.plot(L_val, pow_in_L, label=key)
plt.legend()
plt.xlabel('L')
plt.ylabel(f'Power in mode {var}')
plt.title(f"{key} at {t_val=}M")
plt.yscale('log')

##### Diff in power modes of various levs

In [ ]:
def interpolate_at_time(df: pd.DataFrame,
                        times,
                        time_col: str = "t(M)") -> pd.DataFrame:
    """
    Interpolate all measurement columns in df to the given time(s).

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame. Must contain `time_col` and one or more numeric columns.
    times : scalar or array-like
        The time value(s) at which to interpolate.
    time_col : str, optional
        Name of the time column (default "time").

    Returns
    -------
    pd.DataFrame
        A new DataFrame indexed by the queried time(s), with each
        column interpolated linearly.
    """
    # 1. Set the time column as the index
    df_time = df.set_index(time_col)
    # 2. Build a new index that includes both existing and desired times
    new_idx = df_time.index.union(pd.Index(np.atleast_1d(times)))
    # 3. Reindex to introduce NaNs at query points :contentReference[oaicite:0]{index=0}
    df_reindexed = df_time.reindex(new_idx)
    # 4. Fill those NaNs by linear interpolation along the index :contentReference[oaicite:1]{index=1}
    df_interp = df_reindexed.interpolate(method='polynomial', order=3)
    # 5. Select only the rows at the desired times
    out = df_interp.loc[np.atleast_1d(times)]
    # 6. If the input was a scalar, return a single-row DataFrame
    if np.isscalar(times):
        return out.to_frame().T if isinstance(out, pd.Series) else out
    return out


WT_data_dict.keys()

In [ ]:
var = 'W.dat'
var = 'J.dat'
var = 'DrJ.dat'

base_key = '6_set1_L6s6_250'
base_key = 'BondiCceR0334'

base_pd = WT_data_dict[base_key][var]
t_interp = WT_data_dict[base_key][var]["t(M)"].values
correlation_56 = interpolate_at_time(WT_data_dict[base_key][var], t_interp)

In [ ]:
t_val = 3500
# base_key = 'master_Lev5_R0257'
# base_key = 'master_Lev5_R0257'
# base_key = 'master_Lev5_R0257'

data = WT_data_dict[key][var]
t = data["t(M)"]
closest_index = (t - t_val).abs().idxmin()
base_L_val = []
base_pow_in_L = []
for L in range(16):
    base_L_val.append(L)
    base_pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

for key in WT_data_dict:
    if key == base_key:
        continue
    data = WT_data_dict[key][var]
    t = data["t(M)"]
    closest_index = (t - t_val).abs().idxmin()
    L_val = []
    pow_in_L = []
    for L in range(16):
        L_val.append(L)
        pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

    abs_diff = np.abs(np.array(pow_in_L) - np.array(base_pow_in_L))
    rel_diff = np.abs(np.array(pow_in_L) - np.array(base_pow_in_L)) / np.abs(np.array(base_pow_in_L))
    # plt.scatter(L_val, pow_in_L, label=var)
    # plt.plot(L_val, abs_diff, label=f"abs_diff_{key}")
    plt.plot(L_val, rel_diff, label=f"rel_diff_{key}")
plt.legend()
plt.xlabel('L')
plt.ylabel(f'{var} diff power')
plt.title(f"diff {base_key} at {t_val=}M")
plt.yscale('log')

In [ ]:
t_val = 3500
# base_key1 = 'master_Lev5_R0257'
# base_key = 'master_Lev5_R0257'
# base_key = 'master_Lev5_R0257'

data = WT_data_dict[key][var]
t = data["t(M)"]
closest_index = (t - t_val).abs().idxmin()
base_L_val = []
base_pow_in_L = []
for L in range(16):
    base_L_val.append(L)
    base_pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

for key in WT_data_dict:
    if key == base_key:
        continue
    data = WT_data_dict[key][var]
    t = data["t(M)"]
    closest_index = (t - t_val).abs().idxmin()
    L_val = []
    pow_in_L = []
    for L in range(16):
        L_val.append(L)
        pow_in_L.append(power_in_mode(L, data.iloc[closest_index, :], pre="ReIm"))

    abs_diff = np.abs(np.array(pow_in_L) - np.array(base_pow_in_L))
    rel_diff = np.abs(np.array(pow_in_L) - np.array(base_pow_in_L)) / np.abs(np.array(base_pow_in_L))
    # plt.scatter(L_val, pow_in_L, label=var)
    # plt.plot(L_val, abs_diff, label=f"abs_diff_{key}")
    plt.plot(L_val, rel_diff, label=f"rel_diff_{key}")
plt.legend()
plt.xlabel('L')
plt.ylabel(f'{var} diff power')
plt.title(f"diff {base_key} at {t_val=}M")
plt.yscale('log')

### Power modes wrt time

In [ ]:
WT_data_dict.keys()

In [ ]:
# a = power_in_mode(2,WT_data_dict["6_set1_L6s0_250"]['J.dat'])
pre = "ReIm"
key = "master_Lev5_R0257"
key = "BondiCceR0586"
key = "6_set1_L6s6_100"
var = "J.dat"
for L in range(16):
  t = WT_data_dict[key][var]['t(M)']
  a = power_in_mode(L,WT_data_dict[key][var], pre=pre)
  plt.semilogy(t,np.abs(a),label=f"{L=}")
plt.legend()
plt.title(f"{key}")
plt.xlabel('t(M)')
plt.ylabel(f"{var}: {pre}")

In [ ]:
variables_to_plot = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
# variables_to_plot = ['DrJ.dat', 'J.dat', 'Q.dat', 'R.dat']
variables_to_plot = ['J.dat']

modes_to_plot = ['Re(2,2)','Im(2,2)','Re(2,0)']
modes_to_plot = ['Re(2,2)']
modes_to_plot = ['Re(2,0)']

base_key = 'high_accuracy_Lev5_R0258'
base_key = 'master_Lev5_R0257'
# base_key = '6_set1_L6s6_250'
# base_key = '6_set1_L6s5_250'
# base_key = '6_set1_L3s3_250'
# base_key = '6_set1_L3s2_250'
radius = int(base_key[-3:])

plot_vals = ["_diff","_absdiff","_rel_diff","_rel_absdiff"]
plot_vals = ["_absdiff","_rel_absdiff"]
plot_vals = ["_rel_absdiff"]
# plot_vals = ["_absdiff"]

t_min = -10000
# t_min = 2000
t_min = 2500
t_min = 1190
t_min = 1210
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
# t_max = 3500

avg_len = None
avg_len = 50

num_colors_required = len(list(itertools.product(variables_to_plot,modes_to_plot)))*(len(WT_data_dict)-1)

num_colors_required = 0
for variable,mode,key in itertools.product(variables_to_plot,modes_to_plot,WT_data_dict):
  if key == base_key:
    continue
#   if '250' not in key:
#     continue
  num_colors_required = num_colors_required + 1

if num_colors_required <=10:
  cmap = plt.get_cmap('tab10')
  color = [cmap(i) for i in range(num_colors_required )]
elif num_colors_required <=20:
  cmap = plt.get_cmap('tab20')
  color = [cmap(i) for i in range(num_colors_required )]
else:
  color=plt.cm.hsv(np.linspace(0, 1, num_colors_required))
  color=plt.cm.viridis(np.linspace(0, 1, num_colors_required))

i=-1
plt.close()
for key,variable,mode in itertools.product(WT_data_dict,variables_to_plot,modes_to_plot):
  if key == base_key:
    continue
#   if '250' not in key:
#     continue
  # if not re.search(r"Lev[3,4,5]",key):
  #   continue

  diff_dict = create_diff_dict(WT_data_dict,variable=variable,mode=mode,base_key=base_key)
  t_arr = diff_dict["t(M)"]
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  i = i+1
  for plot_val in plot_vals:
    y_vals = diff_dict[key+plot_val][trimmed_indices]
    if avg_len:
        y_vals = np.convolve(y_vals,np.ones(avg_len),'valid')/avg_len
    x_vals = t_arr[:len(y_vals)]
    match plot_val:
      case "_rel_absdiff":
        plt.semilogy(x_vals,y_vals,label=f"{key}_{variable}_{mode}"+plot_val,linestyle="-",color=color[i])
      case "_absdiff":
        plt.semilogy(x_vals,y_vals,label=f"{key}_{variable}_{mode}"+plot_val,linestyle=":",color=color[i])
        # plt.semilogy(x_vals,y_vals,linestyle=":",color=color[i])
      case "_diff":
        plt.semilogy(x_vals,y_vals,label=f"{key}_{variable}_{mode}"+plot_val,linestyle="--",color=color[i])
      case "_rel_diff":
        plt.semilogy(x_vals,y_vals,label=f"{key}_{variable}_{mode}"+plot_val,linestyle="-.",color=color[i])
    

    plt.xlabel('t')
    plt.ylabel("diff")
    plt.legend()
    plt.tight_layout()
    title = f"Diff {base_key}"
    if avg_len:
      title = title + f" : moving_avg_len={avg_len}"
    plt.title(title)
    # plt.show()
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


### No base diff

In [ ]:
variables_to_plot = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
# variables_to_plot = ['Beta.dat']
variables_to_plot = ['J.dat']

l,m = 2,2
modes_to_plot = ['Re(2,2)','Im(2,2)','Re(2,0)']
modes_to_plot = ['Re(2,2)']

t_min = -10000
# t_min = 2000
t_min = 2500
t_min = 1200
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
t_max = 3500

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

for key in WT_data_dict:
  data_dict = WT_data_dict[key]
  if "s3" not in key:
    continue

  t_arr = data_dict[variables_to_plot[0]]["t(M)"]
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  for i,mode in itertools.product(variables_to_plot, modes_to_plot):
    if labels is None:
      plt.plot(t_arr,data_dict[i][mode][trimmed_indices],label=f"{key}_{i}_{mode}")
    else:
      plt.plot(t_arr,data_dict[i][mode][trimmed_indices],label=f"{labels[key]}_{i}_{mode}")

  plt.xlabel('t')
  plt.legend()
  plt.tight_layout()
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


# WT data modes plots

## load data

In [ ]:
WT_data= {}
# WT_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/BondiCceR0258.h5")
# WT_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev4_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev5_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/BondiCceR0686.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/BondiCceR0100.h5")
# WT_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# WT_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")

# WT_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")

radius = ['0020','0035','0050','0075','0100','0150','0200','0250','0300','0400','0500','0600','0800','1000','1500','2000','2500',]
radius = ['0050','0200','1000','2500']
for r in radius:
    WT_data[f"long_16_2565_10000_14_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14/BondiCceR{r}/BondiCceR{r}.h5")

# levs,run_sets,radius= [],[],[]
# # levs = [0,1,2,3]
# levs = [4,5,6]
# # levs = [0,1,2,3,4,5,6]
# # levs = [3]
# # run_sets = [1,2,3]
# # run_sets = [1,3]
# run_sets = [1]
# radius = [250]
# # radius = [100,150,200,250,300,350,500,700,900]
# # radius = [250,500,900]

# for l,s,r in itertools.product(levs,run_sets,radius):
#   if s == 2 and (l == 0 or l==1):
#     continue
#   if l <= 3:
#     if s == 1:
#       WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")
#     WT_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")
#   else:
#     WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")


# #   WT_data[f"6_set1_L6s3_CAMR_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_CAMR/GW_data_lev3/BondiCceR0{r}/BondiCceR0{r}.h5")
# #   WT_data[f"6_set1_L6s3_min_L_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_L/GW_data_lev3/BondiCceR0{r}/BondiCceR0{r}.h5")


fail_flag=False
for key in WT_data:
  if not WT_data[key].exists():
    fail_flag = True
    print(f"{WT_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
WT_data_dict = {}
for key in WT_data:
  WT_data_dict[key] = WT_to_pandas(WT_data[key])
  print(f"{key} loaded!")

print(WT_data_dict.keys())

In [ ]:
data1 = WT_data_dict['6_set1_L3s0_250']['Beta.dat']
data2 = WT_data_dict['6_set1_L3s3_250']['Beta.dat']
diff = data2 - data1
diff['t(M)'] = data2['t(M)']

In [ ]:
diff = WT_data_dict['6_set1_L3s0_250']['W.dat']

In [ ]:
minT = 0
minT = 1200

maxT = 40000
maxT = 4000
data1 = limit_by_col_val(minT,maxT,'t(M)',data1)
data = limit_by_col_val(minT,maxT,'t(M)',diff)

min_clip_val = 1e-25

domain_col_list = filter_by_regex(regex=["Re"],col_list=data.columns)
domain_col_list = filter_by_regex(regex=[r"Re\(\d6"],col_list=domain_col_list)

visual_data = data[domain_col_list]
visual_data = np.abs(visual_data)
min_val = visual_data.min().min()
print(f"Minimum value in the filtered data: {min_val}")
if min_val < min_clip_val:
  print(f"Min value is too small {min_val}, clipping at {min_clip_val}")
  small_number_mask = visual_data < min_clip_val
  visual_data[small_number_mask] = min_clip_val
visual_data = np.log10(visual_data)

column_names = list(visual_data.columns)

vmin_log,vmax_log = -11,-1
vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()

print(vmin_log,vmax_log)

plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    # cmap='RdYlGn_r', 
    cmap='viridis', 
    origin='lower',
    vmin=vmin_log, 
    vmax=vmax_log
)

plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = len(data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

colorbar = plt.colorbar(imshow_plot)

plt.ylabel('t(M)')
plt.title(f'{key}')
plt.tight_layout()
plt.grid(False)
plt.show()

## Mean/Max of modes over time

In [ ]:
WT_data_dict.keys()

In [ ]:
reduced_dict = {}
L_max_for_mean = 50
variables = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
variables = ['H.dat', 'R.dat']
variables = ['J.dat']
r1 = '100'
# run_keys = [f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L6s5_{r1}']
# run_keys = [f'6_set1_L6s4_{r1}',f'6_set1_L6s5_{r1}',f'6_set1_L6s6_{r1}']
run_keys = [f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
r1='250'
run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
r1='900'
run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
run_keys = WT_data_dict.keys()
# run_keys = ['high_accuracy_Lev0_R0257', 'high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257',]
# run_keys = ['high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258',]

minT = 0
minT = 3200

maxT = 40000
maxT = 4000

reduced_dict = {}
flattened_dict = {}
for run_key in run_keys:

  column_names = None
  reduced_dict[run_key] = {}

  for var in variables:

    data = limit_by_col_val(minT,maxT,'t(M)',WT_data_dict[run_key][var])

    domain_col_list = filter_by_regex(regex=["Re"],col_list=data.columns)
    domain_col_list = filter_by_regex(regex=[r"Re\(\d\d?,\d\d?\)"],col_list=domain_col_list)
    # domain_col_list = filter_by_regex(regex=[r"Re\(\d\d,\d\d?\)"],col_list=domain_col_list)

    if column_names is None and var == "R.dat":
      column_names = domain_col_list

    visual_data = data[domain_col_list]
    visual_data = np.abs(visual_data)
    min_val = visual_data.min().min()
    # print(f"Minimum value in the filtered data: {min_val}")

    reduced_dict[run_key][var] = {
      "mean" : visual_data.mean(),
      "rel_mean" : visual_data.mean()/abs_mean_value_upto_l( visual_data.mean(),L_max_for_mean),
      # "run_mean" : visual_data.mean().cumsum()/range(1,1+len( visual_data.mean())),
      "max" : visual_data.max(),
      "rel_max" : visual_data.max()/abs_mean_value_upto_l( visual_data.max(),L_max_for_mean),
      # "run_max" : visual_data.max().cumsum()/range(1,1+len( visual_data.max())),
    }


  for key,val in reduced_dict[run_key].items():
    for key1,val1 in val.items():
      flattened_dict[f"{key[:-4]}_{key1}_{run_key}"] = val1

sorted_keys = sorted(flattened_dict.keys())
flattened_dict = {key: flattened_dict[key] for key in sorted_keys}

df = None
for key,val in flattened_dict.items():
  if df is None:
    df = pd.DataFrame([val],index=[key])
  else:
    df = pd.concat([df,pd.DataFrame([val],index=[key])])

In [ ]:
dfT = df.T
for col in dfT.columns:
  if "max" not in col:
    continue
  if "rel_" in col:
    continue
  plt.plot(dfT[col],label = col)

plt.title(f'{run_key} t(M)={minT},{maxT}')
plt.tight_layout()
plt.legend()
plt.grid(False)
plt.yscale('log')
plt.xticks([])
plt.show()

In [ ]:
cols = dfT.columns
cols_new = [i for i in cols if "run_max" in i]
a = np.arange(1,len(cols_new)+1)
one_over_x = 2.8e-7/a

plt.plot(dfT[cols_new].loc['Re(16,16)'])
plt.plot(one_over_x)

### Power in modes plots

In [ ]:
reduced_dict = {}
L_max_for_mean = 50
variables = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
# variables = ['H.dat', 'R.dat']
variables = ['J.dat']
# r1 = '100'
# run_keys = [f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L6s5_{r1}']
# run_keys = [f'6_set1_L6s4_{r1}',f'6_set1_L6s5_{r1}',f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='250'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='900'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
run_keys = WT_data_dict.keys()
# run_keys = ['high_accuracy_Lev0_R0257', 'high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257',]
# run_keys = ['high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258',]

minT = 0
minT = 3200

maxT = 40000
maxT = 4000

power_dict = {}
flattened_dict = {}
for run_key in run_keys:

  column_names = None
  power_dict[run_key] = {}

  for var in variables:

    data = limit_by_col_val(minT,maxT,'t(M)',WT_data_dict[run_key][var])

    power_dict[run_key][var] = add_all_L_mode_power(data,16)
    power_dict[run_key]['t(M)'] = data['t(M)'] # same for all vars
    print(run_key,var,len(power_dict[run_key]['t(M)']),f't(M)__{run_key}')
  
  for key,val in power_dict[run_key].items():
    if 't(M)' in key:
      continue
    for key1,val1 in val.items():
      # print(key1)
      if "pow" in key1:
        flattened_dict[f"{key[:-4]}_{key1}__{run_key}"] = val1

sorted_keys = sort_by_power_modes(flattened_dict.keys())
flattened_dict = {key: flattened_dict[key] for key in sorted_keys}
# add_times_for_runs
for run_key in run_keys:
  flattened_dict[f't(M)__{run_key}'] = power_dict[run_key]['t(M)']


In [ ]:

minY,maxY = 1e-40,1e-10
avg_len = 1
# avg_len = 1500

last_val_list = []
r_list = [100,150,200,250,300,350,500,700,900]
# r_list = [250,500,900]
# r_list = [900]
L_list =  [0,1,2,3,4,5,6]
L_list =  [2]
# L_list =  list(range(0,17))
# L_list =  list(range(14,17))

for col in flattened_dict:
  if 'cum_' in col:
    continue
  if 'Re' not in col:
    continue
  # if 't(M)' == col:
  #   continue
  L = get_mode(col)
  r = get_radii(col)
  if "L3" not in col:
    continue
  if (r not in r_list) or (L not in L_list):
    continue
  
  run_name = col.split("__")[1]
  x = flattened_dict[f't(M)__{run_name}'][avg_len-1:]
  y = moving_average_valid(flattened_dict[col],avg_len=avg_len)
  last_val_list.append(y[-1])
  plt.plot(x,y,label = col)

plt.title(f'Power in modes, var={variables}, avg_len={avg_len}')
plt.tight_layout()
plt.legend()
plt.grid(False)
plt.yscale('log')
plt.ylabel(f"Power in modes")
# plt.ylim((minY,maxY))
plt.xlabel("t(M)")
plt.show()

### diff

In [ ]:
WT_data_dict.keys()

In [ ]:
reduced_dict = {}
L_max_for_mean = 50
variables = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
# variables = ['H.dat', 'R.dat']
# variables = ['J.dat']
# r1 = '100'
# run_keys = [f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L6s5_{r1}']
# run_keys = [f'6_set1_L6s4_{r1}',f'6_set1_L6s5_{r1}',f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='250'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='900'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
run_keys = WT_data_dict.keys()
# run_keys = [i for i in run_keys if "L6" in i] + [i for i in run_keys if "L3" in i]
# run_keys = [i for i in run_keys if "900" in i]
run_keys = [i for i in run_keys if "L6" in i]
# run_keys = ['high_accuracy_Lev0_R0257', 'high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257',]
# run_keys = ['high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258',]

# base_key = '6_set1_L6s6_250'
# base_key = 'high_accuracy_Lev5_R0258'
# base_key = 'high_accuracy_Lev2_R0257'
base_key = '6_set1_L6s6_250'
# base_key = '6_set1_L6s3_250'


minT = 0
minT = 3200
minT = 1200

maxT = 40000
maxT = 4000

power_dict = {}
flattened_dict = {}
for run_key in run_keys:
  column_names = None
  power_dict[run_key] = {}

  for var in variables:

    data = limit_by_col_val(minT,maxT,'t(M)',WT_data_dict[run_key][var])

    power_dict[run_key][var] = add_all_L_mode_power(data,16)
    power_dict[run_key]['t(M)'] = data['t(M)'] # same for all vars
  
  for key,val in power_dict[run_key].items():
    if 't(M)' in key:
      continue
    for key1,val1 in val.items():
      # print(key1)
      if "pow" in key1:
        flattened_dict[f"{key[:-4]}_{key1}__{run_key}"] = val1

sorted_keys = sort_by_power_modes(flattened_dict.keys())
flattened_dict = {key: flattened_dict[key] for key in sorted_keys}
# add_times_for_runs
for run_key in run_keys:
  flattened_dict[f't(M)__{run_key}'] = power_dict[run_key]['t(M)']

diff_dict = {}
for var in variables:
  diff_dict[var] = {}
  for pow_mode in power_dict[base_key][var]:
    if 'pow' not in pow_mode:
      continue
    diff_dict[var][pow_mode] = create_power_diff_dict(power_dict,pow_mode,var,base_key)


In [ ]:

# minY,maxY = 1e-40,1e-10
avg_len = 1
avg_len = 800

# minY,maxY = 1e-18,1e-4
r_list = [100,150,200,250,300,350,500,700,900]
# r_list = [250,500,900]
# r_list = [900]
L_list =  [2]
# L_list =  list(range(0,17))
# L_list =  list(range(2,10,2))
# L_list =  list(range(14,17))

plot_vals = ["_diff","_absdiff","_rel_diff","_rel_absdiff"]
plot_vals = ["_absdiff","_rel_absdiff"]
plot_vals = ["_rel_absdiff"]
# plot_vals = ["_absdiff"]

def filter(var,pow_mode,run_name,L_list):
  if run_name == base_key:
    return True
  if 'Re' not in pow_mode:
    return True
  if 'cum' in pow_mode:
    return True
  L = get_mode(pow_mode)
  if (L not in L_list):
    return True
  if run_name == 't(M)':
    return True
  # if not re.search(r"L6s[4,5]",run_name):
  #   return True
#   if not re.search(r"L\ds[1,2,3]_250",run_name):
#     return True
  # if not re.search(r"L\ds[1,2,3,4,5]_250",run_name):
  #   return True
  # if not re.search(r"L6",run_name):
  #   return True
  # if not re.search(r"Lev[4,3]",run_name):
  #   return True
  # if not re.search(r"Lev[0,1,2]",run_name):
  #   return True

num_colors_required = 0
for var in diff_dict.keys():
  for pow_mode in diff_dict[var]:
    for run_name in run_keys:
      if filter(var,pow_mode,run_name,L_list):
        continue
      num_colors_required = num_colors_required + 1

if num_colors_required <=10:
  cmap = plt.get_cmap('tab10')
  color = [cmap(i) for i in range(num_colors_required )]
elif num_colors_required <=20:
  cmap = plt.get_cmap('tab20')
  color = [cmap(i) for i in range(num_colors_required )]
else:
  color=plt.cm.viridis(np.linspace(0, 1, num_colors_required))
  # color=plt.cm.hsv(np.linspace(0, 1, num_colors_required))
  # color=plt.cm.gist_rainbow(np.linspace(0, 1, num_colors_required))

i=0
for var in diff_dict.keys():
  for pow_mode in diff_dict[var]:
    x = diff_dict[var][pow_mode][f't(M)'][avg_len-1:]
    for run_name in run_keys:
      if filter(var,pow_mode,run_name,L_list):
        continue

      # y = moving_average_valid(diff_dict[var][pow_mode][run_name],avg_len=avg_len)
      for plot_val in plot_vals:
        match plot_val:
          case "_rel_absdiff":
            plt.semilogy(x,moving_average_valid(diff_dict[var][pow_mode][run_name+plot_val],avg_len=avg_len),label = f"{pow_mode}_{var} : {run_name}",linestyle="-",color=color[i])
          case "_absdiff":
            if "_rel_absdiff" in plot_vals:
              plt.semilogy(x,moving_average_valid(diff_dict[var][pow_mode][run_name+plot_val],avg_len=avg_len),linestyle=":",color=color[i])
            else:
              plt.semilogy(x,moving_average_valid(diff_dict[var][pow_mode][run_name+plot_val],avg_len=avg_len),label = f"{pow_mode}_{var} : {run_name}",linestyle="-",color=color[i])
          case "_diff":
            plt.semilogy(x,moving_average_valid(diff_dict[var][pow_mode][run_name+plot_val],avg_len=avg_len),label = f"{pow_mode}_{var} : {run_name}",linestyle="--",color=color[i])
          case "_rel_diff":
            plt.semilogy(x,moving_average_valid(diff_dict[var][pow_mode][run_name+plot_val],avg_len=avg_len),label = f"{pow_mode}_{var} : {run_name}",linestyle="-.",color=color[i])
      i = i+1

title = f'Difference in power of L modes from {base_key}, var={variables}, modes={L_list}, avg_len={avg_len}'
if len(title) > 80:
    title = f'Difference in power of L modes from {base_key}, modes={L_list}, avg_len={avg_len}\nvar={variables},'
plt.title(f'Difference in power of L modes from {base_key}, var={variables}, modes={L_list}, avg_len={avg_len}')
plt.tight_layout()
plt.legend()
plt.grid(False)
plt.yscale('log')
plt.ylabel(f"Difference in power of modes")
# plt.ylim((minY,maxY))
plt.xlabel("t(M)")
plt.show()

In [ ]:
from scipy.optimize import curve_fit

one_over_x = last_val_list[0]*1e2/np.array(r_list)
one_over_x2 = last_val_list[0]*1e4/np.array(r_list)**2

# Define the fitting function
def fitting_func(r, a, b):
    return a / r**b
popt, pcov = curve_fit(fitting_func, r_list, last_val_list,p0=[last_val_list[0],2])
a_fit, b_fit = popt

# Create a smooth curve for the fitted function
fit_vals = fitting_func(r_list, a_fit, b_fit)

plt.plot(r_list,last_val_list, label=f"power in mode L={L_list}",marker='x')
# plt.plot(r_list,one_over_x, label="1/x",marker='x')
# plt.plot(r_list,one_over_x2, label="1/(x*x)",marker='x')
plt.plot(r_list,fit_vals,label=f'Fit: a/r^b, a={a_fit:.2e}, b={b_fit:.2f}', linestyle='--')
plt.yscale('log')
plt.legend()
plt.xlabel('r(M)')
plt.ylabel('power in modes')
plt.title(f"L={L_list}")

In [ ]:
flattened_dict['Beta_pow_cum_1_Re_6_set1_L6s6_900']

In [ ]:
dflog10 = df.copy()

# Some values are zero need to deal with those
# min_clip_val = 1e-25
# if min_val < min_clip_val:
#   print(f"Min value in {var} is too small {min_val}, clipping at {min_clip_val}")
#   small_number_mask = visual_data < min_clip_val
#   visual_data[small_number_mask] = min_clip_val
    # visual_data = np.log10(visual_data)

vmin_log,vmax_log = -11,-1
vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = dflog10.min().min()
if vmax_log is None:
  vmax_log = dflog10.max().max()

print(vmin_log,vmax_log)

plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    dflog10[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    # cmap='viridis', 
    origin='lower',
    vmin=vmin_log, 
    vmax=vmax_log
)

plt.xticks(
    ticks=np.arange(len(dflog10.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)


plt.yticks(
    ticks=np.arange(0, len(dflog10.index)), 
    labels=dflog10.index
)

colorbar = plt.colorbar(imshow_plot)

# plt.ylabel('t(M)')
plt.title(f'{run_key} t(M)={minT},{maxT}')
plt.tight_layout()
plt.grid(False)
plt.show()

# Single bh

In [ ]:
cce_data= {}
# cce_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/red_cce.h5")

# levs,run_sets,radius= [],[],[]
# levs = [1,2,3,4,5,6,7,8]
# levs = [1,2,3,4,5,6]
# levs = [5]
# radius = [6,12,25,50,75,100,125,150,175]
# radius = [25,50,75,100]
# radius = [50]
# for l,r in itertools.product(levs,radius):
#   cce_data[f"ode_change_L{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/0/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# levs = [1,2,3,4,5,6,7]
# radius = [3,5,10,25,45,65,95,125,155,190, 290, 390, 490, 590, 690, 790, 890, 990]
# radius = [65]
# for l,r in itertools.product(levs,radius):
#   cce_data[f"1000M_L{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# levs = [1,2,3,4,5,6,7]
# radius = [ 4, 6, 10, 16, 24, 32, 40, 60, 80, 120, 160, 200, 240, 280, 320]
# # radius = [50]
# for l,r in itertools.product(levs,radius):
#   cce_data[f"Lin_L{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# levs = [1,2,3,4,5]
# radius = [3,6,12,25,50,75,100,125,150,175,195,245,295,345]
# radius = [50]
# for l,r in itertools.product(levs,radius):
#   cce_data[f"400M_L{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# levs = [1,2,3,4,5]
# radius = [3,6,12,25,50,75,100,125,150,175,195,245,295,345]
# radius = [50]
# for l,r in itertools.product(levs,radius):
#   cce_data[f"400M_pbc_L{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# outer_bdr = [250,300,350,400,450,500,550,600,650,700,750]
# radius = [3,6,12,25,50,75,100,125,150,175]
# # radius = [50]
# for l,r in itertools.product(outer_bdr,radius):
#   cce_data[f"13_err_fo_{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# outer_bdr = [250,300,350,400,450,500,550,600,650,700,750]
# radius = [3,6,12,25,50,75,100,125,150,175]
# radius = [25,50,75,100,125,150,175]
# # radius = [50]
# for l,r in itertools.product(outer_bdr,radius):
#   cce_data[f"14_err_fo_{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/GW_data/GW_data_lev{l}/BondiCceR{r:04}/red_cce.h5")

# folders = ["Lev5_265","Lev5_265_003","Lev5_265_03","Lev5_265_10","Lev5_265_100","Lev5_265_1000","Lev5_265_10000","Lev5_265_30","Lev5_265_9",]
# folders = ["Lev5_265"]
# radius = [10, 15, 25, 35, 50, 75, 100, 125, 150, 175, 200, 225, 250]
# radius = [50,100]
# for f,r in itertools.product(folders,radius):
#   cce_data[f"18_{f}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/18_zero_spin_const_damp/GW_data/{f}/BondiCceR{r:04}/red_cce.h5")


radius = [20,35,50,75,100,150,200,250,300,400,500,600,800,1000,1500,2000,2500,]
radius = [20,75,250,500,1000,1500,2000,2500,]
radius = [2000]
for r in radius:
  cce_data[f"20_zero_err_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14/BondiCceR{r:04}/red_cce.h5")
#   cce_data[f"20_zero_err_{r}_IC"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14_IC/BondiCceR{r:04}/red_cce.h5")
#   cce_data[f"20_zero_err_{r}_NIR"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14_NIR/BondiCceR{r:04}/red_cce.h5")
#   cce_data[f"20_zero_err_{r}_ZNS"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/GW_data/long_16_2565_10000_14_ZNS/BondiCceR{r:04}/red_cce.h5")

# cce_data[f"temp_50_250"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/17_zero_spin_AMR/GW_data/GW_data_lev0/BondiCceR0100/red_cce.h5")
# fail_flag = False
drop_unavailable_keys = True
unavailable_keys = []
for key in cce_data:
  if not cce_data[key].exists():
    fail_flag = True
    unavailable_keys.append(key)
    print(f"{cce_data[key]} does not exist!")
# if fail_flag:
#   raise Exception("Some paths do not exist!")

for key in unavailable_keys:
  cce_data.pop(key)

In [ ]:

# fail_flag = False
drop_unavailable_keys = True
unavailable_keys = []
for key in cce_data:
  if not cce_data[key].exists():
    fail_flag = True
    unavailable_keys.append(key)
    print(f"{cce_data[key]} does not exist!")
# if fail_flag:
#   raise Exception("Some paths do not exist!")

for key in unavailable_keys:
  cce_data.pop(key)

In [ ]:
t_interpolate = None
t_interpolate = np.linspace(-2000,8000,num=5000)
# t_interpolate = np.linspace(6000,8000,num=1000)
# t_interpolate = np.linspace(-1000,4000,num=5000)

abd_data = {}
failed_keys = {}
for key in cce_data:
  try:
    if "temp" in key:
      abd_data[key] = load_and_pickle(cce_data[key],options = {'t_interpolate':t_interpolate}, reload_data=True)
      abd_data[key] = load_bondi_constraints(cce_data[key])
    elif t_interpolate is None:
      abd_data[key] = load_and_pickle(cce_data[key], reload_data=False)
    #   abd_data[key] = load_and_pickle(cce_data[key],reload_data=True)
      abd_data[key] = load_bondi_constraints(cce_data[key])
    else:
      abd_data[key] = load_and_pickle(cce_data[key],options = {'t_interpolate':t_interpolate}, reload_data=False)
    #   abd_data[key] = load_and_pickle(cce_data[key],reload_data=True)
      abd_data[key] = load_bondi_constraints(cce_data[key])
  except Exception as e:
    failed_keys[key] = str(e)
    print(f"Failed to load and pickle data for key {key}: {e}")
    continue

for key,val in failed_keys.items():
  abd_data.pop(key)
  print(f"{key}: {val}")

print(abd_data.keys())

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [2]

t_min = -10000
# t_min = 2000
# t_min = 2500
# t_min = 4000 - 250
t_max = 10000
# t_max = 7751.5
# t_max = 7000
# t_max = 4000
# t_max = 3500

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

x_data0 = []
y_data0 = []

for key in abd_data:
  violation_dict = abd_data[key]['bondi_violation_norms']
#   radius = int(key.split("_")[-1])
  # if radius > 200:
  #   continue
  # if radius <= 25:
  #   continue
#   if radius != 35:
#     continue
  # if not re.search(r'18_Lev5_265',key):
  #   continue
  # if not re.search(r'10_4000M_CAMR_set1_L6_base_[3,4,5,6]_',key):
  #   continue
  # if not re.search(r'400M_L[1-5]',key):
  #   continue

  t_arr = abd_data[key]["abd"].t
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  for i in bondi_norms_to_plot:
    # x_data0.append(int(key.split("_")[-2]))
    # y_data0.append(violation_dict[i][trimmed_indices][-1])
    if labels is None:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{key}_{i}")
      # plt.semilogy(t_arr,violation_dict[i][trimmed_indices])
    else:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{labels[key]}_{i}")

  plt.xlabel('t(M)')
  plt.ylabel(f"bondi violations {bondi_norms_to_plot}")
  plt.legend()
  plt.tight_layout()
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")

In [ ]:

# x_data0 = x_data0[:-3]
# y_data0 = y_data0[:-3]

plt.semilogy(x_data0,y_data0,marker='x')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# y_data = np.log10(y_data0)
y_data = np.array(y_data0)
x_data = np.array(x_data0)

# Define the linear function
# def linear_func(x, a, b, c, d):
#     return a + b * x + c/x + d/(x*x)
# Define the linear function
def linear_func(x, a, b, c, d):
    return a + b / (x + c)**d

# Perform the fit
popt, pcov = curve_fit(linear_func, x_data, y_data,p0=[1,100,10,5], maxfev=50000)
# popt, pcov = curve_fit(linear_func, x_data, y_data,p0=[1,1,1,6],method='trf', maxfev=5000)
a, b, c, d = popt  # Extract fitted parameters

# Generate fitted y values using the linear function
fitted_y = linear_func(x_data, a, b, c, d)

# Plot the original data
plt.scatter(x_data, y_data, label="Original Data", color="red")

# Plot the fitted line
x_fit = np.linspace(min(x_data), max(x_data), 100)  # Create smooth x values for the fitted line
y_fit = linear_func(x_fit, a, b, c, d)
# plt.plot(x_fit, y_fit, label=f"Fitted Line: y = {a:.3f} + {b:.3f}x + {c:.3f}/x + {d:.3f}/(x*x)", color="blue")
plt.plot(x_fit, y_fit, label=f"Fitted Line: y = {a:.3f} + {b:.3f} / ({c:.3f} + x )**{d:.3f}", color="blue")

# Customize the plot
plt.xlabel("x")
plt.ylabel("y")
plt.yscale('log')
plt.title("Data and Fitted Line")
plt.legend()
plt.grid()

# Show the plot
plt.show()
a,b,c,d

In [ ]:
radius = {
 "50":(1.1581528540505392e-10,
 4415930.873064357,
 43.15124058912971,
 5.846213947609586),
 "75":(4.359321925135716e-11, 86699.62733692401, 6.550548570857155, 5.12793960776798),
  "100" : (9.333520525768315e-11,
 253460.87961644097,
 10.59258112884228,
 5.201469675191469),
 "125":(5.25373272312163e-11, 168652.6696880026, 3.723416133107745, 5.071300081310898),
 "150":(2.3990765414383054e-11,
 181987.48368018138,
 1.4191463150509127,
 5.026842345756565),
 "175":(1.7528319171317524e-11,
 230552.90706779822,
 0.8124681122352008,
 5.015726886364778)
}
radius

### waveforms

In [ ]:
l,m = 2,0
for key in abd_data:
  radius = int(key.split("_")[-1])
  # if radius > 200:
  #   continue
  if radius != 50:
    continue
  # if re.search(r'[6]',key):
  #   continue
  # if "_50_" not in key:
  #   continue
  # if not re.search(r'10_4000M_CAMR_set1_L6_base_[3,4,5,6]_',key):
  #   continue
  if not re.search(r'400M_L[1-5]',key):
    continue
  # if not re.search(r'400M_pbc_L[2-5]',key):
  #   continue
  h = abd_data[key]['abd'].h
  y = h.data[:,  lm(l,m,h.ell_min)]
  t = h.t
  plt.plot(t,y,label=f"{key}_{l}_{m}")

plt.xlabel('t(M)')
plt.ylabel(f"{l}_{m}")
# plt.yscale('log')
plt.legend()
plt.tight_layout()

## Finite radius waveform comparison

In [ ]:
def extract_radii(h5_file_path:Path):
  radii = set()
  with h5py.File(h5_file_path,'r') as f:
    names = []
    f.visit(names.append)
  for name in names:
    if "Version" in name:
      continue
    radii.add(name[1:5])
  radii = list(radii)
  radii.sort()
  return radii

In [ ]:
GW_data_path= {}

levs,run_sets,radius= [],[],[]
levs = [1,2,3,4,5,6,7,8]
levs = [1,2,3,4,5]


for l in levs:
    GW_data_path[f"400M_pbc_L{l}"] = Path(f"/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/GW_data/GW_data_lev{l}/rh_FiniteRadii_CodeUnits")

file = "rh_FiniteRadii_CodeUnits.h5"
for key in GW_data_path:
  GW_data_path[key] = GW_data_path[key]/file
  if not GW_data_path[key].exists():
    raise Exception(f"{key}, {GW_data_path[key]} does not exists!")
radii_list = extract_radii(list(GW_data_path.values())[0])
print(radii_list)


# fail_flag = False
drop_unavailable_keys = True
unavailable_keys = []
for key in cce_data:
  if not cce_data[key].exists():
    fail_flag = True
    unavailable_keys.append(key)
    print(f"{cce_data[key]} does not exist!")
# if fail_flag:
#   raise Exception("Some paths do not exist!")

for key in unavailable_keys:
  cce_data.pop(key)

In [ ]:
radius = radii_list[-1]
# radius = radii_list[4]
GW_data = {}
for key in GW_data_path:
  GW_data[key] = scri.SpEC.read_from_h5(
      f"{GW_data_path[key]}/R{radius}.dir",
      dataType = scri.h,
      frameType = scri.Inertial,
      r_is_scaled_out = True,
      m_is_scaled_out = True,
      )
  print(f"{key} loaded!")

print(GW_data.keys())

In [ ]:

l, m = 2,2
t_max = 2700
t_interpolate = np.linspace(1200+int(radius),t_max+int(radius),num=5000)

base_key = 'master_Lev5'
# base_key = 'master_Lev2'
base_key = 'high_accuracy_Lev5'
# base_key = 'high_accuracy_Lev2'
# base_key = '6_set3_L6s6'
# base_key = '6_set3_L6s5'
# base_key = '6_set1_L3s3'
# base_key = '6_set2_L3s2'
base_h = GW_data[base_key].interpolate(t_interpolate)

def np_abs_func(x):
  return np.abs(np.real(x))
  # return np.abs(x)

for key in GW_data:
  # if 'L6' not in key:
  #   continue
  if not re.search(r'Lev[3,4,5]',key):
    continue
  if key == base_key:
    continue
  h = GW_data[key].interpolate(t_interpolate)
  shited_time = h.t - int(radius)
  diff = h.data[:, lm(l,m,h.ell_min)] - base_h.data[:, lm(l,m,h.ell_min)]
  diff = np_abs_func(diff)
  diff_normalize = diff/np_abs_func(base_h.data[:, lm(l,m,h.ell_min)])
  plt.plot(shited_time,diff,label=f'{key}_{l=}_{m=}')
  # plt.plot(shited_time,diff_normalize,label=f'{key}_rel_{l=}_{m=}')
  # plt.plot(shited_time,base_h.data[:, lm(l,m,h.ell_min)],label=f'{key}_rel2_{l=}_{m=}')
  # plt.plot(shited_time,np_abs_func(base_h.data[:, lm(l,m,h.ell_min)]),label=f'{key}_rel2_{l=}_{m=}')
  # plt.semilogy(h.t,h.energy_flux(),label=f'{key}')

plt.title(f"diff:{base_key}  {l=},{m=}, {radius=}")
plt.xlabel('t')
plt.ylabel("diff")
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/del/LevVariations")/f"L05_finite_radius_{base_key}_{l=}_{m=}_{radius}_{t_max=}.png")

# Rough

In [ ]:
cce_data = Path("/groups/sxs/hchaudha/spec_runs/CCE_stuff/CceExecutables/CharacteristicExtractReduction.h5")
cce_data = Path("/groups/sxs/hchaudha/spec_runs/CCE_stuff/tests/CharacteristicExtractReduction.h5")
cce_data = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/cce_bondi/Lev0_R0257/red_Lev0_R0257_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev3_R0258/red_Lev3_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev4_R0258/red_Lev4_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_R0258/red_Lev5_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/cce_bondi/Lev5_R0257/red_Lev5_R0257_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/red_cce.h5")

In [ ]:

abd = scri.create_abd_from_h5(
      file_name=str(cce_data),
      file_format="spectrecce_v1",
      # ch_mass=1.0,  # Optional; helpful if known
      t_interpolate=np.linspace(-1000,10000,num=5000),  # Optional; for some specified values of `t_worldtube`
      # t_0_superrest=3000,
      # padding_time=400
    )


In [ ]:
h = abd.h
l,m=2,2
# l,m=1,1
print(lm(l,m,h.ell_min))
time_cut = 34500
plt.plot(h.t[h.t<time_cut], h.data[:,  lm(l,m,h.ell_min)][h.t<time_cut])

plt.show()

In [ ]:
violations2 = abd.bianchi_2()
diff_abs = (violations2[1] - violations2[0]).norm()
diff_rel = diff_abs/(violations2[0].norm())
h.data.shape,violations2[0].shape,diff_abs.shape,diff_rel.shape

In [ ]:
plt.semilogy(abd.t,diff_abs,label=f"diff_abs")
plt.semilogy(abd.t,diff_rel,label=f"diff_rel")
plt.ylabel("bianchi2")
plt.xlabel("t")
plt.legend()
plt.tight_layout()

In [ ]:
save_folder = cce_data.parent/"violations"
save_folder.mkdir(exist_ok=True)
for l in range(8):
  for m in range(-l,l+1):
    print(l,m)
    plt.semilogy(abd.t,diff_abs[:, lm(l,m,h.ell_min)],label=f"l,m={l},{m}")
    plt.ylabel("bianchi2")
    plt.xlabel("t")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{save_folder}/l,m={l},{m}.png")
    plt.close()

In [ ]:
save_folder = cce_data.parent/"violations"
save_folder.mkdir(exist_ok=True)
for l in range(8):
  for m in range(-l,l+1):
    print(l,m)
    plt.semilogy(abd.t,diff_rel[:, lm(l,m,h.ell_min)],label=f"l,m={l},{m}")
    plt.ylabel("bianchi2")
    plt.xlabel("t")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{save_folder}/rel_l,m={l},{m}.png")
    plt.close()

In [ ]:
norms = np.linalg.norm(diff, axis=1)

In [ ]:
plt.semilogy(abd.t,norms)

In [ ]:
violations = abd.bondi_violation_norms

In [ ]:
violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
}

# with open(cce_data.parent/"violation_dict.pkl",'wb') as f:
#   pickle.dump(violation_dict,f)

In [ ]:

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='5',color=colors[5])
plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='4',color=colors[4])
plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='3',color=colors[3])
plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2',color=colors[2])
plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='1',color=colors[1])
plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='0',color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()
plt.tight_layout()
plt.savefig(cce_data.parent/"violations.png")

In [ ]:
violations[2]

In [ ]:
violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
}

violation_dict2 = data['Lev5']['0258']

In [ ]:

minT = violation_dict['t'].min()
maxT = violation_dict['t'].max()
print(f"{minT=}, {maxT=}")
mask = (violation_dict2['t'] < maxT) & (violation_dict2['t']  > minT)
# maxT = 200
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict2['t'][mask],violation_dict2['5'][mask],label='5', color=colors[5])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['4'][mask],label='4', color=colors[4])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['3'][mask],label='3', color=colors[3])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['2'][mask],label='2', color=colors[2])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['1'][mask],label='1', color=colors[1])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['0'][mask],label='0', color=colors[0])

plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='2200_5',color=colors[5])
plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='2200_4',color=colors[4])
plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='2200_3',color=colors[3])
plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2200_2',color=colors[2])
plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='2200_1',color=colors[1])
plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='2200_0',color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()

In [ ]:

minT = violation_dict2['t'].min()
maxT = violation_dict2['t'].max()
print(f"{minT=}, {maxT=}")
mask = (violation_dict2['t'] < maxT) & (violation_dict2['t']  > minT)
# maxT = 200
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict2['t'][mask],violation_dict2['5'][mask],label='5', color=colors[5])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['4'][mask],label='4', color=colors[4])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['3'][mask],label='3', color=colors[3])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['2'][mask],label='2', color=colors[2])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['1'][mask],label='1', color=colors[1])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['0'][mask],label='0', color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()

In [ ]:
main_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi")

In [ ]:
bondi_violations_dicts = list(main_folder.glob("*Lev*/bondi*"))
bondi_violations_dicts = list(main_folder.glob("Lev5_R0258/bondi*"))

In [ ]:
bondi_violations_dicts.sort()
bondi_violations_dicts

In [ ]:
import pickle

data = {
  "Lev3":{},
  "Lev4":{},
  "Lev5":{},
  "Lev5_old":{}
}
for file in bondi_violations_dicts:
  with file.open('rb') as f:
    lev = str(file).split("/")[-2][3]
    radius = str(file).split("/")[-2][-4:]
    if radius == "_old":
      lev = lev+"_old"
      radius = str(file).split("/")[-2][-8:-4]
    print(lev,radius)
    data["Lev"+lev][radius] = pickle.load(f)


In [ ]:
levs = data.keys()


bondi_violation = '2'
minT = 220
maxT = 7200
# maxT = 200
RADISU = 200

for lev in levs:
  # if lev!='Lev5':
  #   continue
  for radius in data[lev].keys():
    # if int(radius) != RADISU:
    #   continue
    if int(radius) < 150:
      continue
    # print(radius)

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    elif lev == 'Lev5':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.savefig(f"./{lev}.png")
  plt.close()

for radius in data['Lev5'].keys():
  for lev in levs:
    # if 'Lev5' not in lev:
    #   continue

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    elif lev == 'Lev5':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.tight_layout()
  plt.savefig(f"./{radius}.png")
  plt.close()

In [ ]:
levs = data.keys()


bondi_violation = '2'
minT = 2150
maxT = 7500
RADISU = 900

for lev in levs:
  # if lev!='Lev5':
  #   continue
  for radius in data[lev].keys():
    # if int(radius) != RADISU:
    #   continue
    if int(radius) < 150:
      continue
    # print(radius)

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.savefig(f"./{lev}.png")
  plt.close()

for radius in data['Lev5'].keys():
  for lev in levs:
  # if lev!='Lev5':
  #   continue

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.tight_layout()
  plt.savefig(f"./{int(radius)}.png")
  plt.close()

In [ ]:
main_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations")

In [ ]:
bondi_violations_dicts = list(main_folder.glob("*/bondi*"))
bondi_violations_dicts

In [ ]:
bondi_violations_dicts[0].parent.stem

In [ ]:
import pickle

data = {}
for file in bondi_violations_dicts:
  with file.open('rb') as f:
    data[file.parent.stem] = pickle.load(f)

In [ ]:

bondi_violation = '2'
minT = 215
maxT = 7500
RADISU = 900

for key in data.keys():
  if "test" in key:
    continue
  t = data[key]['t']
  y = data[key][bondi_violation]

  mask = (t < maxT) & (t > minT)
  # plt.plot(t[mask],y[mask],label=f"{key}",linestyle='-.')
  plt.semilogy(t[mask],y[mask],label=f"{key}",linestyle='-.')


plt.xlabel('t')
# plt.ylabel(bondi_violation)
plt.title(f"Bondi Violation {bondi_violation}")
plt.tight_layout()
plt.legend()
plt.savefig(f"./variations.png")
plt.close()

# Compute and save bondi violations

In [ ]:
cce_data_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/GW_data/")

cce_data_paths = list(cce_data_folder.glob(f"**/red_cce.h5"))
for cce_data in cce_data_paths:
  abd = scri.create_abd_from_h5(
        file_name=str(cce_data),
        file_format="spectrecce_v1",
        # ch_mass=1.0,  # Optional; helpful if known
        # t_interpolate=t_worldtube,  # Optional; for some specified values of `t_worldtube`
        # t_0_superrest=3000,
        # padding_time=400
      )

  violations = abd.bondi_violation_norms
  violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
  }

  with open(cce_data.parent/"violation_dict.pkl",'wb') as f:
    pickle.dump(violation_dict,f)


  colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
  plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='2200_5',color=colors[5])
  plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='2200_4',color=colors[4])
  plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='2200_3',color=colors[3])
  plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2200_2',color=colors[2])
  plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='2200_1',color=colors[1])
  plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='2200_0',color=colors[0])

  plt.xlabel('t')
  plt.ylabel("violations")
  plt.legend()
  plt.tight_layout()
  plt.savefig(cce_data.parent/"violations.png")
  plt.close()

In [ ]:
plt.show()

In [ ]:

from spherical_functions import LM_index as lm

In [ ]:
lm(2,2,2)

In [ ]:
h5_file = Path("/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/Lev4_061CCE/BondiCceR0334/red_cce.h5")

with h5py.File(h5_file, 'r') as hf:
    print("Keys: ", list(hf.keys()))
    print("Keys: ", list(hf['SpectreR0334.cce'].keys()))
    print(hf['SpectreR0334.cce']['Psi4'].shape)
    print(hf['SpectreR0334.cce']['EthInertialRetardedTime'].shape)

    data = np.array(hf['SpectreR0334.cce']['Strain'][1:-1:100,:])
    # plt.plot(data)


In [ ]:
t = data[:,0]
y = data[:,9]
# plt.plot(t,y)
# plt.plot(t,data[:,10])
plt.plot(t,np.sqrt(data[:,10]**2+data[:,9]**2))
# plt.yscale('log')

## 1/r sized sphereC

In [ ]:
r_max = 1000
r_min = 54
numC = 15

a = 1.1
GM_sum = (a**numC - 1)/(a-1)
r0 = (r_max - r_min)/GM_sum

# [r0*a**i for i in range(numC)]
size = [r0*a**i for i in range(numC)]
boundaries = [r_min]
for bnd in (np.cumsum([r0*a**i for i in range(numC)]) + r_min):
    boundaries.append(bnd)
boundaries, size
[int(i) for i in boundaries], [int(i) for i in size]